In [1]:
# Ensure these imports are at the top of your script
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')


# ======== CONFIGURABLE PARAMETERS ========\n# Market data parameters
TICKER = 'SPY'  # Main ticker to analyze
VIX_TICKER = '^VIX'  # Volatility index
TNX_TICKER = '^TNX'  # 10-Year Treasury Yield
GLD_TICKER = 'GLD'  # Gold ETF
XLY_TICKER = 'XLY'  # Consumer Discretionary ETF
XLP_TICKER = 'XLP'  # Consumer Staples ETF
XLU_TICKER = 'XLU'  # Utilities ETF
XLF_TICKER = 'XLF'  # Financial ETF
HYG_TICKER = 'HYG'  # High Yield Corporate Bond ETF
TLT_TICKER = 'TLT'  # 20+ Year Treasury Bond ETF
VIX3M_TICKER = '^VIX3M'  # 3-Month VIX
IRX_TICKER = '^IRX'  # 13 Week TBill (~ Short Term Rate Proxy)
UUP_TICKER = 'UUP'  # US Dollar Index ETF
TIP_TICKER = 'TIP'  # TIPS ETF
IEF_TICKER = 'IEF'  # 7-10 Year Treasury ETF
START_DATE = "2018-01-01"  # Historical data start date
#END_DATE = "2025-02-28"    # Data end date (updated to latest)
END_DATE = (datetime.today() + timedelta(days=1)).strftime("%Y-%m-%d")

# Technical indicator parameters
VOL_WINDOW = 21  # Window for volatility calculation (21 days ~ 1 month)
MOMENTUM_WINDOW = 63  # Window for momentum calculation (63 days ~ 3 months)
SMA_FAST = 20  # Fast moving average
SMA_SLOW = 50  # Slow moving average
BB_WINDOW = 20  # Bollinger Bands window
BB_STD = 2  # Bollinger Bands standard deviation multiplier
ATR_WINDOW = 14  # Average True Range window
CHOP_WINDOW = 14  # Choppiness Index window
SECTOR_WINDOW = 10  # Window for sector rotation indicators
CREDIT_MA_WINDOW = 30  # Window for credit spread MA
PERCENTILE_LOOKBACK = 252 # Lookback for calculating rolling percentiles (~1 year)

# Regime Scorecard Configuration
# Define weights, thresholds, percentiles etc. based on the final scorecard design

BULL_REGIME_CONFIG = {
    'rule1_trend_structure': {'weight': 12, 'persistence_days': 5},
    'rule2_price_strength': {'weight': 12, 'thresholds': [1.00, 1.01, 1.02], 'scores': [0.33, 0.67, 1.0]}, # Close > SMA_Slow * threshold
    'rule3_momentum': {'weight': 13, 'percentiles': [0.50, 0.60, 0.70], 'scores': [0.33, 0.67, 1.0]}, # Momentum > Momentum.rolling(252).quantile(percentile)
    'rule4_vol_env': {'weight': 10, 'vix_perc_threshold': 0.40}, # VIX < perc & Vol < 63d mean
    'rule5_credit_cond': {'weight': 12, 'z_thresholds': [0.5, 0.7, 1.0], 'scores': [0.33, 0.67, 1.0]}, # HYG_TLT_Z > z_threshold
    'rule6_broad_part': {'weight': 11, 'adv_perc_threshold': 55}, # AD_Line > 50MA & Adv_Perc > threshold
    'rule7_sector_lead': {'weight': 10, 'xly_xlp_z_threshold': 0.3, 'xlf_spy_z_threshold': 0.0}, # XLY/XLP Z > thresh & XLF/SPY Z > thresh
    'rule8_yield_curve': {'weight': 8, 'spread_threshold': 0.0, 'change_threshold': -0.05}, # Spread > thresh & 21d_Change >= change_thresh
    'rule9_risk_appetite': {'weight': 7, 'bb_perc_b_threshold': 0.7, 'uup_z_threshold': 0.3}, # BB_%B > thresh OR (GLD_Mom > 0 AND UUP_Z < thresh)
    'rule10_vol_confirm': {'weight': 5}, # Volume > 21d mean on up days
}

NEUTRAL_REGIME_CONFIG = {
    'rule1_range_bound': {'weight': 14, 'fast_thresholds': [0.05, 0.03, 0.02], 'slow_thresholds': [0.05, 0.04, 0.02], 'scores': [0.33, 0.67, 1.0]}, # ABS(Price_to_SMA) < threshold
    'rule2_trend_flat': {'weight': 12, 'thresholds': [0.03, 0.02, 0.01], 'scores': [0.33, 0.67, 1.0]}, # ABS(SMA_Ratio - 1) < threshold
    'rule3_choppiness': {'weight': 13, 'lower_threshold': 55, 'upper_threshold': 80}, # Chop Index between thresholds
    'rule4_lim_momentum': {'weight': 11, 'z_thresholds': [1.0, 0.75, 0.5], 'scores': [0.33, 0.67, 1.0]}, # ABS(Momentum_Z) < z_threshold
    'rule5_mod_vol': {'weight': 10, 'z_thresholds': [1.0, 0.8, 0.5], 'scores': [0.33, 0.67, 1.0]}, # ABS(VIX_Z_Score) < z_threshold
    'rule6_band_contract': {'weight': 9, 'lower_z': -0.8, 'upper_z': 0.5}, # BB_Width_Z between lower and upper Z
    'rule7_credit_stab': {'weight': 9, 'z_thresholds': [1.0, 0.7, 0.4], 'scores': [0.33, 0.67, 1.0]}, # ABS(HYG_TLT_Z) < z_threshold
    'rule8_mixed_breadth': {'weight': 8, 'osc_thresholds': [35, 25, 15], 'scores': [0.33, 0.67, 1.0]}, # ABS(McClellan_Oscillator_Norm) < osc_threshold
    'rule9_sector_bal': {'weight': 8, 'xly_xlp_z_threshold': 0.5, 'xlu_spy_z_threshold': 0.5}, # ABS(XLY/XLP_Z) < thresh AND ABS(XLU/SPY_Z) < thresh
    'rule10_mean_reversion': {'weight': 6, 'std_threshold': 0.15, 'mean_lower': 0.4, 'mean_upper': 0.6}, # BB_%B 10d_std > thresh AND 10d_mean between lower/upper
}

BEAR_REGIME_CONFIG = {
    'rule1_trend_structure': {'weight': 12, 'sma_threshold': 0.98, 'persistence_days': 3}, # SMA_Fast < SMA_Slow * thresh for days
    'rule2_price_weakness': {'weight': 12, 'thresholds': [0.99, 0.98, 0.97], 'scores': [0.33, 0.67, 1.0]}, # Close < SMA_Slow * threshold
    'rule3_neg_momentum': {'weight': 13, 'percentiles': [0.40, 0.35, 0.30], 'scores': [0.33, 0.67, 1.0]}, # Momentum < Momentum.rolling(252).quantile(percentile)
    'rule4_elevated_vol': {'weight': 10, 'vix_perc_threshold': 0.70, 'vix_abs_threshold': 20, 'perc_thresholds': [0.60, 0.70, 0.80], 'scores': [0.33, 0.67, 1.0]}, # VIX > perc_thresh AND VIX > abs_thresh; score based on perc
    'rule5_credit_stress': {'weight': 12, 'ratio_threshold': 0.985, 'z_thresholds': [-0.7, -0.9, -1.2], 'scores': [0.33, 0.67, 1.0]}, # Ratio < MA * ratio_thresh AND Z < z_thresh
    'rule6_def_rotation': {'weight': 10, 'xly_xlp_z_threshold': -0.5, 'xlu_spy_z_threshold': 0.3}, # XLY/XLP Z < thresh AND XLU/SPY Z > thresh
    'rule7_breadth_deter': {'weight': 11, 'decl_perc_threshold': 55}, # AD_Line < 50MA AND (Decl_Perc > thresh OR AD_Neg_Div > 0)
    'rule8_vol_structure': {'weight': 8, 'ratio_threshold': 1.03, 'z_threshold': 1.5}, # VIX/VIX3M Ratio > ratio_thresh OR Z > z_thresh
    'rule9_yield_curve_warn': {'weight': 7, 'spread_abs_threshold': 0.05, 'spread_flat_threshold': 0.15, 'change_threshold': -0.10}, # Spread < abs OR (Spread < flat AND 21d_Change < change)
    'rule10_flight_quality': {'weight': 5, 'gld_spy_ratio_threshold': 1.03, 'tlt_threshold': 0.98}, # GLD/SPY Ratio > 63dMean * thresh OR TLT > 20dMax * thresh
}

PERSISTENCE_WEIGHTS = {'t-1': 8, 't-2': 4}

ALL_CONFIG = {
    'Bull': BULL_REGIME_CONFIG,
    'Neutral': NEUTRAL_REGIME_CONFIG,
    'Bear': BEAR_REGIME_CONFIG,
    'Persistence': PERSISTENCE_WEIGHTS
}

# ======== DATA PREPARATION FUNCTIONS ========\n# (Keep load_ad_line_data, download_market_data, calculate_bollinger_bands, calculate_atr, calculate_choppiness_index)
# ... (Keep these functions as they are essential for getting base data and features) ...
def load_ad_line_data(filepath="nyse_breadth_2023.csv"):
    """Load and prepare NYSE breadth data for A/D line indicators"""
    print(f"Loading A/D line data from {filepath}...")
    try:
        ad_data = pd.read_csv(filepath)
        ad_data['Date'] = pd.to_datetime(ad_data['Date'])
        
        # Ensure required columns exist
        required_cols = ['Advancers', 'Decliners', 'Neutral', 'Total Issues']
        if not all(col in ad_data.columns for col in required_cols):
             # Attempt to calculate Total Issues if missing
             if 'Total Issues' not in ad_data.columns and all(c in ad_data.columns for c in ['Advancers', 'Decliners', 'Neutral']):
                 ad_data['Total Issues'] = ad_data['Advancers'] + ad_data['Decliners'] + ad_data['Neutral']
             else:
                raise ValueError(f"Missing required columns in {filepath}. Need: {required_cols}")

        # Calculate basic breadth metrics
        ad_data['Net_Advances'] = ad_data['Advancers'] - ad_data['Decliners']
        ad_data['AD_Line'] = ad_data['Net_Advances'].cumsum()
        
        # A/D Ratio and Z-Score (handle potential division by zero)
        ad_data['AD_Ratio'] = np.where(ad_data['Decliners'] != 0, ad_data['Advancers'] / ad_data['Decliners'], np.nan)
        ad_data['Log_AD_Ratio'] = np.log(ad_data['AD_Ratio'].replace(0, np.nan)) # Avoid log(0)
        ad_data['Log_AD_Ratio_21d_Mean'] = ad_data['Log_AD_Ratio'].rolling(window=21).mean()
        ad_data['Log_AD_Ratio_21d_StdDev'] = ad_data['Log_AD_Ratio'].rolling(window=21).std()
        ad_data['AD_Ratio_Z_Score'] = (ad_data['Log_AD_Ratio'] - ad_data['Log_AD_Ratio_21d_Mean']) / ad_data['Log_AD_Ratio_21d_StdDev']

        # McClellan Oscillator
        ad_data['EMA19_Net_Advances'] = ad_data['Net_Advances'].ewm(span=19, adjust=False).mean()
        ad_data['EMA39_Net_Advances'] = ad_data['Net_Advances'].ewm(span=39, adjust=False).mean()
        ad_data['McClellan_Oscillator'] = ad_data['EMA19_Net_Advances'] - ad_data['EMA39_Net_Advances']
        ad_data['McClellan_Oscillator_Norm'] = np.where(ad_data['Total Issues'] != 0, ad_data['McClellan_Oscillator'] / ad_data['Total Issues'] * 1000, 0)

        # Percentage of Advancing/Declining Issues
        ad_data['Advancing_Percentage'] = np.where(ad_data['Total Issues'] != 0, ad_data['Advancers'] / ad_data['Total Issues'] * 100, 0)
        ad_data['Declining_Percentage'] = np.where(ad_data['Total Issues'] != 0, ad_data['Decliners'] / ad_data['Total Issues'] * 100, 0) # Added for Bear Rule 7
        ad_data['Advancing_Percentage_10MA'] = ad_data['Advancing_Percentage'].rolling(window=10).mean()
        ad_data['Advancing_Percentage_Z'] = (ad_data['Advancing_Percentage'] - ad_data['Advancing_Percentage'].rolling(window=21).mean()) / ad_data['Advancing_Percentage'].rolling(window=21).std()

        # Breadth Thrust Indicator
        ad_data['Daily_Thrust_Denominator'] = ad_data['Advancers'] + ad_data['Decliners']
        ad_data['Daily_Thrust'] = np.where(ad_data['Daily_Thrust_Denominator'] != 0, ad_data['Advancers'] / ad_data['Daily_Thrust_Denominator'], np.nan)
        ad_data['Breadth_Thrust'] = ad_data['Daily_Thrust'].ewm(span=10, adjust=False).mean()
        ad_data['Thrust_Signal'] = (ad_data['Breadth_Thrust'] > 0.65).astype(int)

        # McClellan Summation Index
        ad_data['McClellan_Summation_Index'] = ad_data['McClellan_Oscillator'].cumsum()
        ad_data['McClellan_SI_10MA'] = ad_data['McClellan_Summation_Index'].rolling(window=10).mean()

        # A/D Line Momentum
        ad_data['AD_Line_5d_ROC'] = ad_data['AD_Line'].pct_change(periods=5) * 100
        ad_data['AD_Line_10d_ROC'] = ad_data['AD_Line'].pct_change(periods=10) * 100
        ad_data['AD_Line_20d_ROC'] = ad_data['AD_Line'].pct_change(periods=20) * 100
        ad_data['AD_Line_Momentum_Z'] = (ad_data['AD_Line_10d_ROC'] - ad_data['AD_Line_10d_ROC'].rolling(window=50).mean()) / ad_data['AD_Line_10d_ROC'].rolling(window=50).std()

        # A/D Line Moving Average Crossovers
        ad_data['AD_Line_20MA'] = ad_data['AD_Line'].rolling(window=20).mean()
        ad_data['AD_Line_50MA'] = ad_data['AD_Line'].rolling(window=50).mean()
        ad_data['AD_Line_Golden_Cross'] = (ad_data['AD_Line_20MA'] > ad_data['AD_Line_50MA']).astype(int)
        ad_data['AD_Line_Dist_20MA'] = np.where(ad_data['AD_Line_20MA'] != 0, (ad_data['AD_Line'] / ad_data['AD_Line_20MA'] - 1) * 100, 0)

        # Composite Breadth Indicator (Optional, maybe remove if not used directly)
        # ad_data['AD_Ratio_Z_Norm'] = (ad_data['AD_Ratio_Z_Score'].clip(-3, 3) + 3) / 6
        # ad_data['McClellan_Osc_Norm_Clipped'] = (ad_data['McClellan_Oscillator'].clip(-150, 150) + 150) / 300 # Renamed to avoid confusion
        # ad_data['Breadth_Thrust_Norm'] = ad_data['Breadth_Thrust']
        # ad_data['Advancing_Pct_Norm'] = ad_data['Advancing_Percentage'] / 100
        # ad_data['Composite_Breadth'] = (ad_data['AD_Ratio_Z_Norm'] + ad_data['McClellan_Osc_Norm_Clipped'] + ad_data['Breadth_Thrust_Norm'] + ad_data['Advancing_Pct_Norm']) / 4
        # ad_data['Composite_Breadth_Z'] = (ad_data['Composite_Breadth'] - ad_data['Composite_Breadth'].rolling(window=50).mean()) / ad_data['Composite_Breadth'].rolling(window=50).std().fillna(1)
        
        print("A/D line data loaded and prepared.")
        return ad_data

    except FileNotFoundError:
        print(f"Error: A/D data file not found at {filepath}. Proceeding without breadth data.")
        return None
    except Exception as e:
        print(f"Error loading or processing A/D data from {filepath}: {e}. Proceeding without breadth data.")
        return None


def download_market_data(ticker, vix_ticker, tnx_ticker, gld_ticker, xly_ticker, xlp_ticker, xlu_ticker, xlf_ticker, hyg_ticker, tlt_ticker, vix3m_ticker, irx_ticker, uup_ticker, tip_ticker, ief_ticker, start_date, end_date=None):
    """Download and prepare market data for all required tickers."""
    if end_date is None:
        end_date = (datetime.today() + timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"Downloading market data from {start_date} to {end_date}...")

    tickers_to_download = {
        'main': ticker, 'VIX': vix_ticker, 'TNX': tnx_ticker, 'GLD': gld_ticker,
        'XLY': xly_ticker, 'XLP': xlp_ticker, 'XLU': xlu_ticker, 'XLF': xlf_ticker,
        'HYG': hyg_ticker, 'TLT': tlt_ticker, 'VIX3M': vix3m_ticker, 'IRX': irx_ticker,
        'UUP': uup_ticker, 'TIP': tip_ticker, 'IEF': ief_ticker
    }

    data_frames = {}
    for name, symbol in tickers_to_download.items():
        print(f"Downloading {name} ({symbol})...")
        df = yf.download(symbol, start=start_date, end=end_date, auto_adjust=(name == 'main'), progress=False) # auto_adjust only for main ticker usually

        # Standardize columns (handle multi-index)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(1) # Assumes second level is ticker, adjust if needed

        df = df.reset_index()
        
        # Select relevant columns and rename
        if name == 'main':
            df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
        elif name in ['VIX', 'TNX', 'GLD', 'XLY', 'XLP', 'XLU', 'XLF', 'HYG', 'TLT', 'VIX3M', 'IRX', 'UUP', 'TIP', 'IEF']:
             # Use Adj Close if available and differs from Close, otherwise use Close
             close_col = 'Adj Close' if ('Adj Close' in df.columns and not df['Close'].equals(df['Adj Close'])) else 'Close'
             df = df[['Date', close_col]].rename(columns={close_col: name})
        else:
             # Fallback for unexpected case
             df = df[['Date', 'Close']].rename(columns={'Close': name})

        data_frames[name] = df

    # Merge all dataframes
    print("Merging dataframes...")
    df_merged = data_frames['main']
    for name, df in data_frames.items():
        if name != 'main':
            df_merged = pd.merge(df_merged, df, on='Date', how='left')

    # Forward fill missing values for auxiliary tickers
    print("Forward filling missing values...")
    cols_to_ffill = ['VIX', 'TNX', 'GLD', 'XLY', 'XLP', 'XLU', 'XLF', 'HYG', 'TLT', 'VIX3M', 'IRX', 'UUP', 'TIP', 'IEF']
    for col in cols_to_ffill:
        if col in df_merged.columns:
            df_merged[col] = df_merged[col].fillna(method='ffill')
        else:
             print(f"Warning: Expected column '{col}' not found after download/merge.")


    # Basic log return calculation
    df_merged['Log_Return'] = np.log(df_merged['Close'] / df_merged['Close'].shift(1)) * 100
    if 'GLD' in df_merged.columns:
       df_merged['GLD_Log_Return'] = np.log(df_merged['GLD'] / df_merged['GLD'].shift(1)) * 100
    else:
        df_merged['GLD_Log_Return'] = 0 # Or np.nan if preferred

    print("Market data download and initial preparation complete.")
    return df_merged


def calculate_bollinger_bands(df, window=20, num_std=2):
    """Calculate Bollinger Bands and related metrics"""
    df['BB_Middle'] = df['Close'].rolling(window=window).mean()
    rolling_std = df['Close'].rolling(window=window).std()
    df['BB_Upper'] = df['BB_Middle'] + (rolling_std * num_std)
    df['BB_Lower'] = df['BB_Middle'] - (rolling_std * num_std)
    df['BB_Width'] = np.where(df['BB_Middle'] != 0, (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle'] * 100, 0)
    df['BB_PercentB'] = np.where((df['BB_Upper'] - df['BB_Lower']) != 0, (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower']), 0.5) # Default to 0.5 if width is 0
    return df

def calculate_atr(df, window=14):
    """Calculate Average True Range (ATR)"""
    df['TR1'] = abs(df['High'] - df['Low'])
    df['TR2'] = abs(df['High'] - df['Close'].shift(1))
    df['TR3'] = abs(df['Low'] - df['Close'].shift(1))
    df['True_Range'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    # Use EMA for ATR calculation as is standard
    df['ATR'] = df['True_Range'].ewm(alpha=1/window, adjust=False).mean()
    df['ATR_Normalized'] = np.where(df['Close'] != 0, df['ATR'] / df['Close'] * 100, 0)
    df = df.drop(['TR1', 'TR2', 'TR3', 'True_Range'], axis=1)
    return df

def calculate_choppiness_index(df, window=14):
    """Calculate Choppiness Index"""
    if 'ATR' not in df.columns:
        # ATR calculation dependency
        df = calculate_atr(df, window=window) # Use same window for ATR dependency
        
    log10_window = np.log10(window)
    df['MaxHi'] = df['High'].rolling(window=window).max()
    df['MinLo'] = df['Low'].rolling(window=window).min()
    df['ATR_Sum'] = df['ATR'].rolling(window=window).sum() # Sum of ATR over the window
    
    # Calculate Choppiness Index, handle potential division by zero or log(0)
    range_val = df['MaxHi'] - df['MinLo']
    df['Choppiness_Index'] = np.where(
        (range_val > 0) & (df['ATR_Sum'] > 0), # Ensure arguments to log10 are positive
        100 * np.log10(df['ATR_Sum'] / range_val) / log10_window,
        50 # Default to 50 (mid-point) if calculation fails
    )
    df = df.drop(['MaxHi', 'MinLo', 'ATR_Sum'], axis=1, errors='ignore')
    return df

def calculate_features(data, ad_data=None, percentile_lookback=PERCENTILE_LOOKBACK):
    """Calculate ALL features for regime classification, including A/D line indicators and rolling percentiles."""
    print("Calculating technical features...")
    df = data.copy()

    # --- Basic Price & Trend ---
    df['Momentum'] = df['Close'].pct_change(periods=MOMENTUM_WINDOW) * 100
    df['SMA_Fast'] = df['Close'].rolling(window=SMA_FAST).mean()
    df['SMA_Slow'] = df['Close'].rolling(window=SMA_SLOW).mean()
    df['SMA_Ratio'] = np.where(df['SMA_Slow'] != 0, df['SMA_Fast'] / df['SMA_Slow'], 1)
    df['Price_to_SMA_Fast'] = np.where(df['SMA_Fast'] != 0, df['Close'] / df['SMA_Fast'] - 1, 0)
    df['Price_to_SMA_Slow'] = np.where(df['SMA_Slow'] != 0, df['Close'] / df['SMA_Slow'] - 1, 0)

    # --- Volatility & Risk ---
    df['Volatility'] = df['Log_Return'].rolling(window=VOL_WINDOW).std() * np.sqrt(252) # Annualized
    df = calculate_bollinger_bands(df, window=BB_WINDOW, num_std=BB_STD)
    df = calculate_atr(df, window=ATR_WINDOW)
    df = calculate_choppiness_index(df, window=CHOP_WINDOW)
    if 'VIX' in df.columns:
        df['VIX_Z_Score'] = (df['VIX'] - df['VIX'].rolling(window=VOL_WINDOW).mean()) / df['VIX'].rolling(window=VOL_WINDOW).std()

    if 'VIX' in df.columns and 'VIX3M' in df.columns:
        df['VIX_VIX3M_Ratio'] = np.where(df['VIX3M'] != 0, df['VIX'] / df['VIX3M'], 1)
        df['VIX_VIX3M_Ratio_Z'] = (df['VIX_VIX3M_Ratio'] - df['VIX_VIX3M_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['VIX_VIX3M_Ratio'].rolling(window=VOL_WINDOW).std()
    
    df['BB_Width_Z'] = (df['BB_Width'] - df['BB_Width'].rolling(window=VOL_WINDOW).mean()) / df['BB_Width'].rolling(window=VOL_WINDOW).std()


    # --- Intermarket & Macro ---
    # Yield Curve
    if 'TNX' in df.columns and 'IRX' in df.columns:
        df['Yield_Curve_Spread'] = df['TNX'] - df['IRX']
        df['Yield_Curve_Spread_Change'] = df['Yield_Curve_Spread'].diff(21) # Using 21d change per rules
    
    # Gold
    if 'GLD' in df.columns:
        df['GLD_Momentum'] = df['GLD'].pct_change(periods=MOMENTUM_WINDOW) * 100
        df['GLD_Z_Score'] = (df['GLD'] - df['GLD'].rolling(window=VOL_WINDOW).mean()) / df['GLD'].rolling(window=VOL_WINDOW).std()
        df['GLD_SPY_Ratio'] = np.where(df['Close'] != 0, df['GLD'] / df['Close'], np.nan)
        df['GLD_SPY_Ratio_63d_Mean'] = df['GLD_SPY_Ratio'].rolling(window=63).mean() # For Bear Rule 10

    # Sector Ratios & Z-Scores
    if 'XLY' in df.columns and 'XLP' in df.columns:
        df['XLY_XLP_Ratio'] = np.where(df['XLP'] != 0, df['XLY'] / df['XLP'], np.nan)
        df['XLY_XLP_Z'] = (df['XLY_XLP_Ratio'] - df['XLY_XLP_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLY_XLP_Ratio'].rolling(window=VOL_WINDOW).std()

    if 'XLU' in df.columns:
        df['XLU_SPY_Ratio'] = np.where(df['Close'] != 0, df['XLU'] / df['Close'], np.nan)
        df['XLU_SPY_Z'] = (df['XLU_SPY_Ratio'] - df['XLU_SPY_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLU_SPY_Ratio'].rolling(window=VOL_WINDOW).std()

    if 'XLF' in df.columns:
        df['XLF_SPY_Ratio'] = np.where(df['Close'] != 0, df['XLF'] / df['Close'], np.nan)
        df['XLF_SPY_Z'] = (df['XLF_SPY_Ratio'] - df['XLF_SPY_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['XLF_SPY_Ratio'].rolling(window=VOL_WINDOW).std()

    # Credit Spread (HYG/TLT)
    if 'HYG' in df.columns and 'TLT' in df.columns:
        df['HYG_TLT_Ratio'] = np.where(df['TLT'] != 0, df['HYG'] / df['TLT'], np.nan)
        df['HYG_TLT_MA'] = df['HYG_TLT_Ratio'].rolling(window=CREDIT_MA_WINDOW).mean()
        df['HYG_TLT_Z'] = (df['HYG_TLT_Ratio'] - df['HYG_TLT_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['HYG_TLT_Ratio'].rolling(window=VOL_WINDOW).std()

    # Dollar (UUP)
    if 'UUP' in df.columns:
         df['UUP_Z_Score'] = (df['UUP'] - df['UUP'].rolling(window=VOL_WINDOW).mean()) / df['UUP'].rolling(window=VOL_WINDOW).std()

    # TIPS (Inflation Expectations Proxy)
    # if 'TIP' in df.columns and 'IEF' in df.columns:
    #     df['TIP_IEF_Ratio'] = np.where(df['IEF'] != 0, df['TIP'] / df['IEF'], np.nan)
    #     df['TIP_IEF_Ratio_Z'] = (df['TIP_IEF_Ratio'] - df['TIP_IEF_Ratio'].rolling(window=VOL_WINDOW).mean()) / df['TIP_IEF_Ratio'].rolling(window=VOL_WINDOW).std()

    # Treasuries for Flight-to-Quality Rule
    if 'TLT' in df.columns:
        df['TLT_20d_Max'] = df['TLT'].rolling(window=20).max()


    # --- Volume ---
    df['Volume_MA21'] = df['Volume'].rolling(window=21).mean()


    # --- Rolling Percentiles (needed for graduated scoring) ---
    print("Calculating rolling percentiles...")
    df['Momentum_Perc_50'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.50)
    df['Momentum_Perc_60'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.60)
    df['Momentum_Perc_70'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.70)
    df['Momentum_Perc_40'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.40)
    df['Momentum_Perc_35'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.35)
    df['Momentum_Perc_30'] = df['Momentum'].rolling(window=percentile_lookback).quantile(0.30)

    if 'VIX' in df.columns:
        df['VIX_Perc_40'] = df['VIX'].rolling(window=percentile_lookback).quantile(0.40)
        df['VIX_Perc_60'] = df['VIX'].rolling(window=percentile_lookback).quantile(0.60)
        df['VIX_Perc_70'] = df['VIX'].rolling(window=percentile_lookback).quantile(0.70)
        df['VIX_Perc_80'] = df['VIX'].rolling(window=percentile_lookback).quantile(0.80)

    if 'HYG_TLT_Z' in df.columns:
        # Calculate required Z-score percentiles if needed (or use fixed thresholds directly)
        # For now, using fixed Z thresholds from config
        pass


    # --- Add A/D LINE FEATURES if available ---
    if ad_data is not None and not ad_data.empty:
        print("Merging A/D line features...")
        # Select specific columns needed for rules
        ad_cols_to_merge = [
            'Date', 'AD_Line', 'AD_Line_50MA', 'Advancing_Percentage', 'Declining_Percentage',
            'McClellan_Oscillator_Norm' # Renamed earlier
        ]
        # Check if AD_Negative_Divergence was calculated and add it
        # Example calculation (needs refinement based on exact definition)
        price_new_high = df['Close'] > df['Close'].rolling(window=20).max().shift(1)
        if 'AD_Line' in ad_data.columns:
            ad_line_not_confirming = ad_data['AD_Line'] < ad_data['AD_Line'].rolling(window=20).max().shift(1)
            ad_data['AD_Negative_Divergence'] = (price_new_high & ad_line_not_confirming).astype(int)
            if 'AD_Negative_Divergence' not in ad_cols_to_merge:
                ad_cols_to_merge.append('AD_Negative_Divergence')


        # Ensure 'Date' is datetime in both dataframes before merge
        df['Date'] = pd.to_datetime(df['Date'])
        ad_data['Date'] = pd.to_datetime(ad_data['Date'])

        # Filter ad_data to only necessary columns before merge
        ad_data_subset = ad_data[[col for col in ad_cols_to_merge if col in ad_data.columns]].copy()

        # Merge
        df = pd.merge(df, ad_data_subset, on='Date', how='left')
        # Note: Ffill might not be appropriate for all AD features, use with caution or handle NaNs in scoring
        # df[ad_cols_to_merge[1:]] = df[ad_cols_to_merge[1:]].fillna(method='ffill') # Optional ffill
    else:
        print("Skipping A/D line feature merge as data is not available.")
        # Add placeholder columns if AD data is missing to avoid errors in scoring functions
        placeholder_ad_cols = ['AD_Line', 'AD_Line_50MA', 'Advancing_Percentage', 'Declining_Percentage', 'McClellan_Oscillator_Norm', 'AD_Negative_Divergence']
        for col in placeholder_ad_cols:
            if col not in df.columns:
                df[col] = np.nan


    # --- Final Cleanup ---
    # Drop rows with NaNs essential for core calculations (e.g., SMAs)
    # Be careful not to drop too much if percentiles have long lookbacks
    initial_len = len(df)
    required_cols_for_calc = ['SMA_Slow', 'Momentum', 'Volatility', 'BB_Width_Z'] # Add more if critical
    df = df.dropna(subset=required_cols_for_calc).reset_index(drop=True)
    print(f"Dropped {initial_len - len(df)} rows due to NaNs in essential features.")

    print("Feature calculation complete.")
    return df

# ======== REGIME SCORING FUNCTIONS ========\n
# ======== ENHANCED REGIME SCORING FUNCTIONS ========

def get_graduated_score(value, thresholds, scores, ascending=True):
    """Helper function for graduated scoring."""
    if ascending: # Higher value gets higher score (e.g., Momentum > threshold)
        for i in range(len(thresholds) - 1, -1, -1):
            if value > thresholds[i]:
                return scores[i]
    else: # Lower value gets higher score (e.g., ABS(Z-score) < threshold)
        for i in range(len(thresholds)):
            if value < thresholds[i]:
                return scores[i]
    return 0 # Default score if no threshold is met


def score_bull_regime(data_row, config, store_details=True):
    """Calculates the raw score for the Bull regime for a given data row with detailed rule tracking."""
    score = 0
    weight_sum = 0 
    rule_details = {} if store_details else None

    # Handle potential errors if data is missing for a day
    try:
        # Rule 1: Trend Structure
        weight = config['rule1_trend_structure']['weight']
        weight_sum += weight
        rule1_passed = pd.notna(data_row['SMA_Fast']) and pd.notna(data_row['SMA_Slow']) and data_row['SMA_Fast'] > data_row['SMA_Slow']
        rule1_score = weight if rule1_passed else 0
        score += rule1_score

        if store_details:
            rule_details['rule1_trend_structure'] = {
                'name': 'Trend Structure',
                'description': 'SMA Fast > SMA Slow (persistence checked separately)',
                'weight': weight,
                'passed': rule1_passed,
                'score': rule1_score,
                'values': {
                    'SMA_Fast': data_row['SMA_Fast'] if pd.notna(data_row['SMA_Fast']) else 0,
                    'SMA_Slow': data_row['SMA_Slow'] if pd.notna(data_row['SMA_Slow']) else 0,
                    'Persistence': data_row.get('bull_trend_persistence', 'N/A')
                }
            }

        # Rule 2: Price Strength
        weight = config['rule2_price_strength']['weight']
        weight_sum += weight
        rule2_passed = False
        rule2_score = 0
        rule2_ratio = 0
        
        if pd.notna(data_row['Close']) and pd.notna(data_row['SMA_Slow']) and data_row['SMA_Slow'] > 0:
            rule2_ratio = data_row['Close'] / data_row['SMA_Slow']
            thresholds = config['rule2_price_strength']['thresholds']
            scores = config['rule2_price_strength']['scores']
            rule2_score = weight * get_graduated_score(rule2_ratio, thresholds, scores, ascending=True)
            rule2_passed = rule2_score > 0
            score += rule2_score

        if store_details:
            rule_details['rule2_price_strength'] = {
                'name': 'Price Strength',
                'description': 'Close > SMA_Slow * threshold (graduated)',
                'weight': weight,
                'passed': rule2_passed,
                'score': rule2_score,
                'values': {
                    'Close': data_row['Close'] if pd.notna(data_row['Close']) else 0,
                    'SMA_Slow': data_row['SMA_Slow'] if pd.notna(data_row['SMA_Slow']) else 0,
                    'Ratio': rule2_ratio,
                    'Thresholds': config['rule2_price_strength']['thresholds']
                }
            }

        # Rule 3: Medium-Term Momentum
        weight = config['rule3_momentum']['weight']
        weight_sum += weight
        rule3_passed = False
        rule3_score = 0
        
        if pd.notna(data_row['Momentum']):
             percentiles = [data_row.get(f'Momentum_Perc_{int(p*100)}', np.nan) for p in config['rule3_momentum']['percentiles']]
             if not any(pd.isna(p) for p in percentiles):
                 scores = config['rule3_momentum']['scores']
                 rule3_score = weight * get_graduated_score(data_row['Momentum'], percentiles, scores, ascending=True)
                 rule3_passed = rule3_score > 0
                 score += rule3_score

        if store_details:
            rule_details['rule3_momentum'] = {
                'name': 'Medium-Term Momentum',
                'description': 'Momentum > historical percentile thresholds',
                'weight': weight,
                'passed': rule3_passed,
                'score': rule3_score,
                'values': {
                    'Momentum': data_row['Momentum'] if pd.notna(data_row['Momentum']) else 0,
                    'Percentile_50': data_row.get('Momentum_Perc_50', 'N/A'),
                    'Percentile_60': data_row.get('Momentum_Perc_60', 'N/A'),
                    'Percentile_70': data_row.get('Momentum_Perc_70', 'N/A')
                }
            }

        # Rule 4: Volatility Environment
        weight = config['rule4_vol_env']['weight']
        weight_sum += weight
        vix_threshold_perc = config['rule4_vol_env']['vix_perc_threshold']
        vix_perc_col = f'VIX_Perc_{int(vix_threshold_perc*100)}'
        
        rule4_passed = (pd.notna(data_row['VIX']) and pd.notna(data_row[vix_perc_col]) and 
                       data_row['VIX'] < data_row[vix_perc_col] and
                       pd.notna(data_row['Volatility']) and pd.notna(data_row['Volatility_63d_Mean']) and 
                       data_row['Volatility'] < data_row['Volatility_63d_Mean'])
        
        rule4_score = weight if rule4_passed else 0
        score += rule4_score

        if store_details:
            rule_details['rule4_vol_env'] = {
                'name': 'Volatility Environment',
                'description': f'VIX < {vix_threshold_perc*100}th percentile AND Volatility < 63d mean',
                'weight': weight,
                'passed': rule4_passed,
                'score': rule4_score,
                'values': {
                    'VIX': data_row['VIX'] if pd.notna(data_row['VIX']) else 0,
                    f'VIX_{int(vix_threshold_perc*100)}th_Percentile': data_row.get(vix_perc_col, 'N/A'),
                    'Volatility': data_row['Volatility'] if pd.notna(data_row['Volatility']) else 0,
                    'Volatility_63d_Mean': data_row['Volatility_63d_Mean'] if pd.notna(data_row['Volatility_63d_Mean']) else 0
                }
            }

        # Rule 5: Credit Conditions
        weight = config['rule5_credit_cond']['weight']
        weight_sum += weight
        rule5_passed = False
        rule5_score = 0
        
        if pd.notna(data_row['HYG_TLT_Z']):
            thresholds = config['rule5_credit_cond']['z_thresholds']
            scores = config['rule5_credit_cond']['scores']
            rule5_score = weight * get_graduated_score(data_row['HYG_TLT_Z'], thresholds, scores, ascending=True)
            rule5_passed = rule5_score > 0
            score += rule5_score
        
        if store_details:
            rule_details['rule5_credit_cond'] = {
                'name': 'Credit Conditions',
                'description': 'HYG/TLT Z-score > threshold (graduated)',
                'weight': weight,
                'passed': rule5_passed,
                'score': rule5_score,
                'values': {
                    'HYG_TLT_Z': data_row['HYG_TLT_Z'] if pd.notna(data_row['HYG_TLT_Z']) else 0,
                    'Thresholds': config['rule5_credit_cond']['z_thresholds']
                }
            }
        
        # Rule 6: Broad Participation
        weight = config['rule6_broad_part']['weight']
        weight_sum += weight
        adv_perc_threshold = config['rule6_broad_part']['adv_perc_threshold']
        
        rule6_passed = (pd.notna(data_row['AD_Line']) and pd.notna(data_row['AD_Line_50MA']) and 
                       data_row['AD_Line'] > data_row['AD_Line_50MA'] and
                       pd.notna(data_row['Advancing_Percentage']) and 
                       data_row['Advancing_Percentage'] > adv_perc_threshold)
        
        rule6_score = weight if rule6_passed else 0
        score += rule6_score

        if store_details:
            rule_details['rule6_broad_part'] = {
                'name': 'Broad Participation',
                'description': f'AD Line > 50MA AND Advancing % > {adv_perc_threshold}%',
                'weight': weight,
                'passed': rule6_passed,
                'score': rule6_score,
                'values': {
                    'AD_Line': data_row['AD_Line'] if pd.notna(data_row['AD_Line']) else 'N/A',
                    'AD_Line_50MA': data_row['AD_Line_50MA'] if pd.notna(data_row['AD_Line_50MA']) else 'N/A',
                    'Advancing_Percentage': data_row['Advancing_Percentage'] if pd.notna(data_row['Advancing_Percentage']) else 0,
                    'Threshold': adv_perc_threshold
                }
            }

        # Rule 7: Sector Leadership
        weight = config['rule7_sector_lead']['weight']
        weight_sum += weight
        xly_xlp_z_thresh = config['rule7_sector_lead']['xly_xlp_z_threshold']
        xlf_spy_z_thresh = config['rule7_sector_lead']['xlf_spy_z_threshold']
        
        rule7_passed = (pd.notna(data_row['XLY_XLP_Z']) and data_row['XLY_XLP_Z'] > xly_xlp_z_thresh and
                       pd.notna(data_row['XLF_SPY_Z']) and data_row['XLF_SPY_Z'] > xlf_spy_z_thresh)
        
        rule7_score = weight if rule7_passed else 0
        score += rule7_score

        if store_details:
            rule_details['rule7_sector_lead'] = {
                'name': 'Sector Leadership',
                'description': f'XLY/XLP Z > {xly_xlp_z_thresh} AND XLF/SPY Z > {xlf_spy_z_thresh}',
                'weight': weight,
                'passed': rule7_passed,
                'score': rule7_score,
                'values': {
                    'XLY_XLP_Z': data_row['XLY_XLP_Z'] if pd.notna(data_row['XLY_XLP_Z']) else 0,
                    'XLF_SPY_Z': data_row['XLF_SPY_Z'] if pd.notna(data_row['XLF_SPY_Z']) else 0,
                    'XLY_XLP_Threshold': xly_xlp_z_thresh,
                    'XLF_SPY_Threshold': xlf_spy_z_thresh
                }
            }

        # Rule 8: Yield Curve Health
        weight = config['rule8_yield_curve']['weight']
        weight_sum += weight
        spread_thresh = config['rule8_yield_curve']['spread_threshold']
        change_thresh = config['rule8_yield_curve']['change_threshold']
        
        rule8_passed = (pd.notna(data_row['Yield_Curve_Spread']) and 
                       data_row['Yield_Curve_Spread'] > spread_thresh and
                       pd.notna(data_row['Yield_Curve_Spread_Change']) and 
                       data_row['Yield_Curve_Spread_Change'] >= change_thresh)
        
        rule8_score = weight if rule8_passed else 0
        score += rule8_score

        if store_details:
            rule_details['rule8_yield_curve'] = {
                'name': 'Yield Curve Health',
                'description': f'Spread > {spread_thresh} AND 21d Change >= {change_thresh}',
                'weight': weight,
                'passed': rule8_passed,
                'score': rule8_score,
                'values': {
                    'Yield_Curve_Spread': data_row['Yield_Curve_Spread'] if pd.notna(data_row['Yield_Curve_Spread']) else 0,
                    'Yield_Curve_Spread_Change': data_row['Yield_Curve_Spread_Change'] if pd.notna(data_row['Yield_Curve_Spread_Change']) else 0,
                    'Spread_Threshold': spread_thresh,
                    'Change_Threshold': change_thresh
                }
            }

        # Rule 9: Risk Appetite
        weight = config['rule9_risk_appetite']['weight']
        weight_sum += weight
        bb_thresh = config['rule9_risk_appetite']['bb_perc_b_threshold']
        uup_z_thresh = config['rule9_risk_appetite']['uup_z_threshold']
        
        rule9_cond1 = pd.notna(data_row['BB_PercentB']) and data_row['BB_PercentB'] > bb_thresh
        rule9_cond2 = (pd.notna(data_row['GLD_Momentum']) and data_row['GLD_Momentum'] > 0 and
                      pd.notna(data_row['UUP_Z_Score']) and data_row['UUP_Z_Score'] < uup_z_thresh)
        
        rule9_passed = rule9_cond1 or rule9_cond2
        rule9_score = weight if rule9_passed else 0
        score += rule9_score

        if store_details:
            rule_details['rule9_risk_appetite'] = {
                'name': 'Risk Appetite',
                'description': f'BB %B > {bb_thresh} OR (GLD Mom > 0 AND UUP Z < {uup_z_thresh})',
                'weight': weight,
                'passed': rule9_passed,
                'score': rule9_score,
                'values': {
                    'BB_PercentB': data_row['BB_PercentB'] if pd.notna(data_row['BB_PercentB']) else 0,
                    'GLD_Momentum': data_row['GLD_Momentum'] if pd.notna(data_row['GLD_Momentum']) else 0,
                    'UUP_Z_Score': data_row['UUP_Z_Score'] if pd.notna(data_row['UUP_Z_Score']) else 0,
                    'BB_Threshold': bb_thresh,
                    'UUP_Threshold': uup_z_thresh,
                    'Condition1_Met': rule9_cond1,
                    'Condition2_Met': rule9_cond2
                }
            }

        # Rule 10: Volume Confirmation
        weight = config['rule10_vol_confirm']['weight']
        weight_sum += weight
        
        rule10_passed = (pd.notna(data_row['Close']) and pd.notna(data_row['Close_Shift1']) and 
                        data_row['Close'] > data_row['Close_Shift1'] and
                        pd.notna(data_row['Volume']) and pd.notna(data_row['Volume_MA21']) and 
                        data_row['Volume'] > data_row['Volume_MA21'])
        
        rule10_score = weight if rule10_passed else 0
        score += rule10_score

        if store_details:
            rule_details['rule10_vol_confirm'] = {
                'name': 'Volume Confirmation',
                'description': 'Volume > 21d MA on up days',
                'weight': weight,
                'passed': rule10_passed,
                'score': rule10_score,
                'values': {
                    'Close': data_row['Close'] if pd.notna(data_row['Close']) else 0,
                    'Previous_Close': data_row['Close_Shift1'] if pd.notna(data_row['Close_Shift1']) else 0,
                    'Volume': data_row['Volume'] if pd.notna(data_row['Volume']) else 0,
                    'Volume_MA21': data_row['Volume_MA21'] if pd.notna(data_row['Volume_MA21']) else 0,
                    'Up_Day': data_row['Close'] > data_row['Close_Shift1'] if pd.notna(data_row['Close']) and pd.notna(data_row['Close_Shift1']) else False
                }
            }

    except KeyError as e:
        if store_details:
            rule_details['error'] = f"Missing key {e} in data_row for Bull scoring"
        pass
    except Exception as e:
        if store_details:
            rule_details['error'] = f"Error scoring Bull regime: {e}"
        print(f"Error scoring Bull regime for date {data_row.get('Date', 'Unknown')}: {e}")

    # Ensure score doesn't exceed maximum possible weight
    score = min(score, 100) 
    return score, rule_details


def score_neutral_regime(data_row, config, store_details=True):
    """Calculates the raw score for the Neutral regime for a given data row with detailed rule tracking."""
    score = 0
    weight_sum = 0
    rule_details = {} if store_details else None

    try:
        # Rule 1: Range-Bound Price
        weight = config['rule1_range_bound']['weight']
        weight_sum += weight
        rule1_passed = False
        rule1_score = 0
        
        if pd.notna(data_row['Price_to_SMA_Fast']) and pd.notna(data_row['Price_to_SMA_Slow']):
            fast_thresh = config['rule1_range_bound']['fast_thresholds']
            slow_thresh = config['rule1_range_bound']['slow_thresholds']
            scores = config['rule1_range_bound']['scores']
            
            score_fast = get_graduated_score(abs(data_row['Price_to_SMA_Fast']), fast_thresh, scores, ascending=False)
            score_slow = get_graduated_score(abs(data_row['Price_to_SMA_Slow']), slow_thresh, scores, ascending=False)
            
            rule1_score = weight * min(score_fast, score_slow)
            rule1_passed = rule1_score > 0
            score += rule1_score

        if store_details:
            rule_details['rule1_range_bound'] = {
                'name': 'Range-Bound Price',
                'description': 'ABS(Price to SMA) < thresholds for both fast and slow',
                'weight': weight,
                'passed': rule1_passed,
                'score': rule1_score,
                'values': {
                    'Price_to_SMA_Fast': abs(data_row['Price_to_SMA_Fast']) if pd.notna(data_row['Price_to_SMA_Fast']) else 0,
                    'Price_to_SMA_Slow': abs(data_row['Price_to_SMA_Slow']) if pd.notna(data_row['Price_to_SMA_Slow']) else 0,
                    'Fast_Thresholds': config['rule1_range_bound']['fast_thresholds'],
                    'Slow_Thresholds': config['rule1_range_bound']['slow_thresholds']
                }
            }

        # Rule 2: Trend Flatness
        weight = config['rule2_trend_flat']['weight']
        weight_sum += weight
        rule2_passed = False
        rule2_score = 0
        
        if pd.notna(data_row['SMA_Ratio']):
            thresholds = config['rule2_trend_flat']['thresholds']
            scores = config['rule2_trend_flat']['scores']
            rule2_score = weight * get_graduated_score(abs(data_row['SMA_Ratio'] - 1), thresholds, scores, ascending=False)
            rule2_passed = rule2_score > 0
            score += rule2_score
            
        if store_details:
            rule_details['rule2_trend_flat'] = {
                'name': 'Trend Flatness',
                'description': 'ABS(SMA_Ratio - 1) < threshold (graduated)',
                'weight': weight,
                'passed': rule2_passed,
                'score': rule2_score,
                'values': {
                    'SMA_Ratio': data_row['SMA_Ratio'] if pd.notna(data_row['SMA_Ratio']) else 0,
                    'SMA_Ratio_Deviation': abs(data_row['SMA_Ratio'] - 1) if pd.notna(data_row['SMA_Ratio']) else 0,
                    'Thresholds': config['rule2_trend_flat']['thresholds']
                }
            }
            
        # Rule 3: Choppiness
        weight = config['rule3_choppiness']['weight']
        weight_sum += weight
        lower_thresh = config['rule3_choppiness']['lower_threshold']
        upper_thresh = config['rule3_choppiness']['upper_threshold']
        
        rule3_passed = pd.notna(data_row['Choppiness_Index']) and lower_thresh < data_row['Choppiness_Index'] < upper_thresh
        rule3_score = weight if rule3_passed else 0
        score += rule3_score

        if store_details:
            rule_details['rule3_choppiness'] = {
                'name': 'Choppiness',
                'description': f'Choppiness Index between {lower_thresh} and {upper_thresh}',
                'weight': weight,
                'passed': rule3_passed,
                'score': rule3_score,
                'values': {
                    'Choppiness_Index': data_row['Choppiness_Index'] if pd.notna(data_row['Choppiness_Index']) else 0,
                    'Lower_Threshold': lower_thresh,
                    'Upper_Threshold': upper_thresh
                }
            }

        # Rule 4: Limited Momentum
        weight = config['rule4_lim_momentum']['weight']
        weight_sum += weight
        rule4_passed = False
        rule4_score = 0
        
        if pd.notna(data_row['Momentum_Z']):
            thresholds = config['rule4_lim_momentum']['z_thresholds']
            scores = config['rule4_lim_momentum']['scores']
            rule4_score = weight * get_graduated_score(abs(data_row['Momentum_Z']), thresholds, scores, ascending=False)
            rule4_passed = rule4_score > 0
            score += rule4_score

        if store_details:
            rule_details['rule4_lim_momentum'] = {
                'name': 'Limited Momentum',
                'description': 'ABS(Momentum Z-Score) < threshold (graduated)',
                'weight': weight,
                'passed': rule4_passed,
                'score': rule4_score,
                'values': {
                    'Momentum_Z': data_row['Momentum_Z'] if pd.notna(data_row['Momentum_Z']) else 0,
                    'Momentum_Z_Absolute': abs(data_row['Momentum_Z']) if pd.notna(data_row['Momentum_Z']) else 0,
                    'Thresholds': config['rule4_lim_momentum']['z_thresholds']
                }
            }

        # Rule 5: Moderate Volatility
        weight = config['rule5_mod_vol']['weight']
        weight_sum += weight
        rule5_passed = False
        rule5_score = 0
        
        if pd.notna(data_row['VIX_Z_Score']):
             thresholds = config['rule5_mod_vol']['z_thresholds']
             scores = config['rule5_mod_vol']['scores']
             rule5_score = weight * get_graduated_score(abs(data_row['VIX_Z_Score']), thresholds, scores, ascending=False)
             rule5_passed = rule5_score > 0
             score += rule5_score

        if store_details:
            rule_details['rule5_mod_vol'] = {
                'name': 'Moderate Volatility',
                'description': 'ABS(VIX Z-Score) < threshold (graduated)',
                'weight': weight,
                'passed': rule5_passed,
                'score': rule5_score,
                'values': {
                    'VIX_Z_Score': data_row['VIX_Z_Score'] if pd.notna(data_row['VIX_Z_Score']) else 0,
                    'VIX_Z_Absolute': abs(data_row['VIX_Z_Score']) if pd.notna(data_row['VIX_Z_Score']) else 0,
                    'Thresholds': config['rule5_mod_vol']['z_thresholds']
                }
            }

        # Rule 6: Band Contraction
        weight = config['rule6_band_contract']['weight']
        weight_sum += weight
        lower_z = config['rule6_band_contract']['lower_z']
        upper_z = config['rule6_band_contract']['upper_z']
        
        rule6_passed = pd.notna(data_row['BB_Width_Z']) and lower_z < data_row['BB_Width_Z'] < upper_z
        rule6_score = weight if rule6_passed else 0
        score += rule6_score

        if store_details:
            rule_details['rule6_band_contract'] = {
                'name': 'Band Contraction',
                'description': f'BB Width Z-Score between {lower_z} and {upper_z}',
                'weight': weight,
                'passed': rule6_passed,
                'score': rule6_score,
                'values': {
                    'BB_Width_Z': data_row['BB_Width_Z'] if pd.notna(data_row['BB_Width_Z']) else 0,
                    'Lower_Z': lower_z,
                    'Upper_Z': upper_z
                }
            }

        # Rule 7: Credit Market Stability
        weight = config['rule7_credit_stab']['weight']
        weight_sum += weight
        rule7_passed = False
        rule7_score = 0
        
        if pd.notna(data_row['HYG_TLT_Z']):
             thresholds = config['rule7_credit_stab']['z_thresholds']
             scores = config['rule7_credit_stab']['scores']
             rule7_score = weight * get_graduated_score(abs(data_row['HYG_TLT_Z']), thresholds, scores, ascending=False)
             rule7_passed = rule7_score > 0
             score += rule7_score

        if store_details:
            rule_details['rule7_credit_stab'] = {
                'name': 'Credit Market Stability',
                'description': 'ABS(HYG/TLT Z-Score) < threshold (graduated)',
                'weight': weight,
                'passed': rule7_passed,
                'score': rule7_score,
                'values': {
                    'HYG_TLT_Z': data_row['HYG_TLT_Z'] if pd.notna(data_row['HYG_TLT_Z']) else 0,
                    'HYG_TLT_Z_Absolute': abs(data_row['HYG_TLT_Z']) if pd.notna(data_row['HYG_TLT_Z']) else 0,
                    'Thresholds': config['rule7_credit_stab']['z_thresholds']
                }
            }

        # Rule 8: Mixed Breadth
        weight = config['rule8_mixed_breadth']['weight']
        weight_sum += weight
        rule8_passed = False
        rule8_score = 0
        
        if pd.notna(data_row['McClellan_Oscillator_Norm']):
            thresholds = config['rule8_mixed_breadth']['osc_thresholds']
            scores = config['rule8_mixed_breadth']['scores']
            rule8_score = weight * get_graduated_score(abs(data_row['McClellan_Oscillator_Norm']), thresholds, scores, ascending=False)
            rule8_passed = rule8_score > 0
            score += rule8_score

        if store_details:
            rule_details['rule8_mixed_breadth'] = {
                'name': 'Mixed Breadth',
                'description': 'ABS(McClellan Oscillator Normalized) < threshold (graduated)',
                'weight': weight,
                'passed': rule8_passed,
                'score': rule8_score,
                'values': {
                    'McClellan_Oscillator_Norm': data_row['McClellan_Oscillator_Norm'] if pd.notna(data_row['McClellan_Oscillator_Norm']) else 0,
                    'McClellan_Absolute': abs(data_row['McClellan_Oscillator_Norm']) if pd.notna(data_row['McClellan_Oscillator_Norm']) else 0,
                    'Thresholds': config['rule8_mixed_breadth']['osc_thresholds']
                }
            }

        # Rule 9: Sector Balance
        weight = config['rule9_sector_bal']['weight']
        weight_sum += weight
        xly_xlp_z_thresh = config['rule9_sector_bal']['xly_xlp_z_threshold']
        xlu_spy_z_thresh = config['rule9_sector_bal']['xlu_spy_z_threshold']
        
        rule9_passed = (pd.notna(data_row['XLY_XLP_Z']) and abs(data_row['XLY_XLP_Z']) < xly_xlp_z_thresh and
                       pd.notna(data_row['XLU_SPY_Z']) and abs(data_row['XLU_SPY_Z']) < xlu_spy_z_thresh)
        
        rule9_score = weight if rule9_passed else 0
        score += rule9_score

        if store_details:
            rule_details['rule9_sector_bal'] = {
                'name': 'Sector Balance',
                'description': f'ABS(XLY/XLP Z) < {xly_xlp_z_thresh} AND ABS(XLU/SPY Z) < {xlu_spy_z_thresh}',
                'weight': weight,
                'passed': rule9_passed,
                'score': rule9_score,
                'values': {
                    'XLY_XLP_Z': data_row['XLY_XLP_Z'] if pd.notna(data_row['XLY_XLP_Z']) else 0,
                    'XLY_XLP_Z_Absolute': abs(data_row['XLY_XLP_Z']) if pd.notna(data_row['XLY_XLP_Z']) else 0,
                    'XLU_SPY_Z': data_row['XLU_SPY_Z'] if pd.notna(data_row['XLU_SPY_Z']) else 0,
                    'XLU_SPY_Z_Absolute': abs(data_row['XLU_SPY_Z']) if pd.notna(data_row['XLU_SPY_Z']) else 0,
                    'XLY_XLP_Threshold': xly_xlp_z_thresh,
                    'XLU_SPY_Threshold': xlu_spy_z_thresh
                }
            }

        # Rule 10: Mean-Reversion Character
        weight = config['rule10_mean_reversion']['weight']
        weight_sum += weight
        std_thresh = config['rule10_mean_reversion']['std_threshold']
        mean_lower = config['rule10_mean_reversion']['mean_lower']
        mean_upper = config['rule10_mean_reversion']['mean_upper']
        
        rule10_passed = (pd.notna(data_row['BB_PercentB_10d_Std']) and data_row['BB_PercentB_10d_Std'] > std_thresh and
                        pd.notna(data_row['BB_PercentB_10d_Mean']) and mean_lower < data_row['BB_PercentB_10d_Mean'] < mean_upper)
        
        rule10_score = weight if rule10_passed else 0
        score += rule10_score

        if store_details:
            rule_details['rule10_mean_reversion'] = {
                'name': 'Mean-Reversion Character',
                'description': f'BB %B 10d StdDev > {std_thresh} AND 10d Mean between {mean_lower}-{mean_upper}',
                'weight': weight,
                'passed': rule10_passed,
                'score': rule10_score,
                'values': {
                    'BB_PercentB_10d_Std': data_row['BB_PercentB_10d_Std'] if pd.notna(data_row['BB_PercentB_10d_Std']) else 0,
                    'BB_PercentB_10d_Mean': data_row['BB_PercentB_10d_Mean'] if pd.notna(data_row['BB_PercentB_10d_Mean']) else 0,
                    'StdDev_Threshold': std_thresh,
                    'Mean_Lower': mean_lower,
                    'Mean_Upper': mean_upper
                }
            }

    except KeyError as e:
        if store_details:
            rule_details['error'] = f"Missing key {e} in data_row for Neutral scoring"
        pass
    except Exception as e:
        if store_details:
            rule_details['error'] = f"Error scoring Neutral regime: {e}"
        print(f"Error scoring Neutral regime for date {data_row.get('Date', 'Unknown')}: {e}")

    return min(score, 100), rule_details


def score_bear_regime(data_row, config, store_details=True):
    """Calculates the raw score for the Bear regime for a given data row with detailed rule tracking."""
    score = 0
    weight_sum = 0
    rule_details = {} if store_details else None

    try:
        # Rule 1: Trend Structure
        weight = config['rule1_trend_structure']['weight']
        weight_sum += weight
        sma_thresh = config['rule1_trend_structure']['sma_threshold']
        
        rule1_passed = (pd.notna(data_row['SMA_Fast']) and pd.notna(data_row['SMA_Slow']) and 
                       data_row['SMA_Slow'] > 0 and data_row['SMA_Fast'] < (data_row['SMA_Slow'] * sma_thresh))
        
        rule1_score = weight if rule1_passed else 0
        score += rule1_score

        if store_details:
            rule_details['rule1_trend_structure'] = {
                'name': 'Trend Structure',
                'description': f'SMA Fast < SMA Slow * {sma_thresh} (persistence checked separately)',
                'weight': weight,
                'passed': rule1_passed,
                'score': rule1_score,
                'values': {
                    'SMA_Fast': data_row['SMA_Fast'] if pd.notna(data_row['SMA_Fast']) else 0,
                    'SMA_Slow': data_row['SMA_Slow'] if pd.notna(data_row['SMA_Slow']) else 0,
                    'Threshold': sma_thresh,
                    'Persistence': data_row.get('bear_trend_persistence', 'N/A')
                }
            }

        # Rule 2: Price Weakness
        weight = config['rule2_price_weakness']['weight']
        weight_sum += weight
        rule2_passed = False
        rule2_score = 0
        rule2_ratio = 0
        
        if pd.notna(data_row['Close']) and pd.notna(data_row['SMA_Slow']) and data_row['SMA_Slow'] > 0:
            rule2_ratio = data_row['Close'] / data_row['SMA_Slow']
            thresholds = config['rule2_price_weakness']['thresholds']
            scores = config['rule2_price_weakness']['scores']
            rule2_score = weight * get_graduated_score(rule2_ratio, thresholds, scores, ascending=False)
            rule2_passed = rule2_score > 0
            score += rule2_score

        if store_details:
            rule_details['rule2_price_weakness'] = {
                'name': 'Price Weakness',
                'description': 'Close < SMA_Slow * threshold (graduated)',
                'weight': weight,
                'passed': rule2_passed,
                'score': rule2_score,
                'values': {
                    'Close': data_row['Close'] if pd.notna(data_row['Close']) else 0,
                    'SMA_Slow': data_row['SMA_Slow'] if pd.notna(data_row['SMA_Slow']) else 0,
                    'Ratio': rule2_ratio,
                    'Thresholds': config['rule2_price_weakness']['thresholds']
                }
            }

        # Rule 3: Negative Momentum
        weight = config['rule3_neg_momentum']['weight']
        weight_sum += weight
        rule3_passed = False
        rule3_score = 0
        
        if pd.notna(data_row['Momentum']):
             percentiles = [data_row.get(f'Momentum_Perc_{int(p*100)}', np.nan) for p in config['rule3_neg_momentum']['percentiles']]
             if not any(pd.isna(p) for p in percentiles):
                 scores = config['rule3_neg_momentum']['scores']
                 rule3_score = weight * get_graduated_score(data_row['Momentum'], percentiles, scores, ascending=False)
                 rule3_passed = rule3_score > 0
                 score += rule3_score

        if store_details:
            rule_details['rule3_neg_momentum'] = {
                'name': 'Negative Momentum',
                'description': 'Momentum < historical percentile thresholds',
                'weight': weight,
                'passed': rule3_passed,
                'score': rule3_score,
                'values': {
                    'Momentum': data_row['Momentum'] if pd.notna(data_row['Momentum']) else 0,
                    'Percentile_40': data_row.get('Momentum_Perc_40', 'N/A'),
                    'Percentile_35': data_row.get('Momentum_Perc_35', 'N/A'),
                    'Percentile_30': data_row.get('Momentum_Perc_30', 'N/A')
                }
            }

        # Rule 4: Elevated Volatility
        weight = config['rule4_elevated_vol']['weight']
        weight_sum += weight
        vix_perc_thresh = config['rule4_elevated_vol']['vix_perc_threshold']
        vix_abs_thresh = config['rule4_elevated_vol']['vix_abs_threshold']
        vix_perc_col = f'VIX_Perc_{int(vix_perc_thresh*100)}'
        
        rule4_basic_passed = (pd.notna(data_row['VIX']) and pd.notna(data_row[vix_perc_col]) and 
                             data_row['VIX'] > data_row[vix_perc_col] and data_row['VIX'] > vix_abs_thresh)
        
        rule4_score = 0
        rule4_passed = False
        
        if rule4_basic_passed:
            perc_levels = [data_row.get(f'VIX_Perc_{int(p*100)}', np.nan) for p in config['rule4_elevated_vol']['perc_thresholds']]
            if not any(pd.isna(p) for p in perc_levels):
                 scores = config['rule4_elevated_vol']['scores']
                 rule4_score = weight * get_graduated_score(data_row['VIX'], perc_levels, scores, ascending=True)
                 rule4_passed = rule4_score > 0
                 score += rule4_score

        if store_details:
            rule_details['rule4_elevated_vol'] = {
                'name': 'Elevated Volatility',
                'description': f'VIX > {vix_perc_thresh*100}th percentile AND VIX > {vix_abs_thresh} (graduated)',
                'weight': weight,
                'passed': rule4_passed,
                'score': rule4_score,
                'values': {
                    'VIX': data_row['VIX'] if pd.notna(data_row['VIX']) else 0,
                    f'VIX_{int(vix_perc_thresh*100)}th_Percentile': data_row.get(vix_perc_col, 'N/A'),
                    'VIX_Absolute_Threshold': vix_abs_thresh,
                    'Basic_Condition_Met': rule4_basic_passed,
                    'Percentile_Levels': config['rule4_elevated_vol']['perc_thresholds'] if 'perc_thresholds' in config['rule4_elevated_vol'] else 'N/A'
                }
            }

        # Rule 5: Credit Stress
        weight = config['rule5_credit_stress']['weight']
        weight_sum += weight
        ratio_thresh = config['rule5_credit_stress']['ratio_threshold']
        rule5_passed = False
        rule5_score = 0
        
        rule5_basic_passed = (pd.notna(data_row['HYG_TLT_Ratio']) and pd.notna(data_row['HYG_TLT_MA']) and 
                             data_row['HYG_TLT_MA'] > 0 and data_row['HYG_TLT_Ratio'] < (data_row['HYG_TLT_MA'] * ratio_thresh) and 
                             pd.notna(data_row['HYG_TLT_Z']))
        
        if rule5_basic_passed:
            thresholds = config['rule5_credit_stress']['z_thresholds']
            scores = config['rule5_credit_stress']['scores']
            rule5_score = weight * get_graduated_score(data_row['HYG_TLT_Z'], thresholds, scores, ascending=False)
            rule5_passed = rule5_score > 0
            score += rule5_score

        if store_details:
            rule_details['rule5_credit_stress'] = {
                'name': 'Credit Stress',
                'description': f'HYG/TLT Ratio < MA * {ratio_thresh} AND Z < threshold (graduated)',
                'weight': weight,
                'passed': rule5_passed,
                'score': rule5_score,
                'values': {
                    'HYG_TLT_Ratio': data_row['HYG_TLT_Ratio'] if pd.notna(data_row['HYG_TLT_Ratio']) else 0,
                    'HYG_TLT_MA': data_row['HYG_TLT_MA'] if pd.notna(data_row['HYG_TLT_MA']) else 0,
                    'HYG_TLT_Z': data_row['HYG_TLT_Z'] if pd.notna(data_row['HYG_TLT_Z']) else 0,
                    'Ratio_Threshold': ratio_thresh,
                    'Basic_Condition_Met': rule5_basic_passed,
                    'Z_Thresholds': config['rule5_credit_stress']['z_thresholds']
                }
            }

        # Rule 6: Defensive Rotation
        weight = config['rule6_def_rotation']['weight']
        weight_sum += weight
        xly_xlp_z_thresh = config['rule6_def_rotation']['xly_xlp_z_threshold']
        xlu_spy_z_thresh = config['rule6_def_rotation']['xlu_spy_z_threshold']
        
        rule6_passed = (pd.notna(data_row['XLY_XLP_Z']) and data_row['XLY_XLP_Z'] < xly_xlp_z_thresh and
                       pd.notna(data_row['XLU_SPY_Z']) and data_row['XLU_SPY_Z'] > xlu_spy_z_thresh)
        
        rule6_score = weight if rule6_passed else 0
        score += rule6_score

        if store_details:
            rule_details['rule6_def_rotation'] = {
                'name': 'Defensive Rotation',
                'description': f'XLY/XLP Z < {xly_xlp_z_thresh} AND XLU/SPY Z > {xlu_spy_z_thresh}',
                'weight': weight,
                'passed': rule6_passed,
                'score': rule6_score,
                'values': {
                    'XLY_XLP_Z': data_row['XLY_XLP_Z'] if pd.notna(data_row['XLY_XLP_Z']) else 0,
                    'XLU_SPY_Z': data_row['XLU_SPY_Z'] if pd.notna(data_row['XLU_SPY_Z']) else 0,
                    'XLY_XLP_Threshold': xly_xlp_z_thresh,
                    'XLU_SPY_Threshold': xlu_spy_z_thresh
                }
            }

        # Rule 7: Breadth Deterioration
        weight = config['rule7_breadth_deter']['weight']
        weight_sum += weight
        decl_perc_thresh = config['rule7_breadth_deter']['decl_perc_threshold']
        
        cond1 = pd.notna(data_row['AD_Line']) and pd.notna(data_row['AD_Line_50MA']) and data_row['AD_Line'] < data_row['AD_Line_50MA']
        cond2 = pd.notna(data_row['Declining_Percentage']) and data_row['Declining_Percentage'] > decl_perc_thresh
        cond3 = pd.notna(data_row['AD_Negative_Divergence']) and data_row['AD_Negative_Divergence'] > 0
        
        rule7_passed = cond1 and (cond2 or cond3)
        rule7_score = weight if rule7_passed else 0
        score += rule7_score

        if store_details:
            rule_details['rule7_breadth_deter'] = {
                'name': 'Breadth Deterioration',
                'description': f'AD Line < 50MA AND (Declining % > {decl_perc_thresh} OR Neg Divergence)',
                'weight': weight,
                'passed': rule7_passed,
                'score': rule7_score,
                'values': {
                    'AD_Line': data_row['AD_Line'] if pd.notna(data_row['AD_Line']) else 'N/A',
                    'AD_Line_50MA': data_row['AD_Line_50MA'] if pd.notna(data_row['AD_Line_50MA']) else 'N/A',
                    'Declining_Percentage': data_row['Declining_Percentage'] if pd.notna(data_row['Declining_Percentage']) else 0,
                    'AD_Negative_Divergence': data_row['AD_Negative_Divergence'] if pd.notna(data_row['AD_Negative_Divergence']) else 0,
                    'Condition1_Met': cond1,
                    'Condition2_Met': cond2,
                    'Condition3_Met': cond3
                }
            }

        # Rule 8: Volatility Structure
        weight = config['rule8_vol_structure']['weight']
        weight_sum += weight
        ratio_thresh = config['rule8_vol_structure']['ratio_threshold']
        z_thresh = config['rule8_vol_structure']['z_threshold']
        
        cond1_vol = pd.notna(data_row['VIX_VIX3M_Ratio']) and data_row['VIX_VIX3M_Ratio'] > ratio_thresh
        cond2_vol = pd.notna(data_row['VIX_VIX3M_Ratio_Z']) and data_row['VIX_VIX3M_Ratio_Z'] > z_thresh
        
        rule8_passed = cond1_vol or cond2_vol
        rule8_score = weight if rule8_passed else 0
        score += rule8_score

        if store_details:
            rule_details['rule8_vol_structure'] = {
                'name': 'Volatility Structure',
                'description': f'VIX/VIX3M Ratio > {ratio_thresh} OR Z > {z_thresh}',
                'weight': weight,
                'passed': rule8_passed,
                'score': rule8_score,
                'values': {
                    'VIX_VIX3M_Ratio': data_row['VIX_VIX3M_Ratio'] if pd.notna(data_row['VIX_VIX3M_Ratio']) else 0,
                    'VIX_VIX3M_Ratio_Z': data_row['VIX_VIX3M_Ratio_Z'] if pd.notna(data_row['VIX_VIX3M_Ratio_Z']) else 0,
                    'Ratio_Threshold': ratio_thresh,
                    'Z_Threshold': z_thresh,
                    'Condition1_Met': cond1_vol,
                    'Condition2_Met': cond2_vol
                }
            }

        # Rule 9: Yield Curve Warning
        weight = config['rule9_yield_curve_warn']['weight']
        weight_sum += weight
        spread_abs_thresh = config['rule9_yield_curve_warn']['spread_abs_threshold']
        spread_flat_thresh = config['rule9_yield_curve_warn']['spread_flat_threshold']
        change_thresh = config['rule9_yield_curve_warn']['change_threshold']
        
        cond1_yc = pd.notna(data_row['Yield_Curve_Spread']) and data_row['Yield_Curve_Spread'] < spread_abs_thresh
        cond2_yc = (pd.notna(data_row['Yield_Curve_Spread']) and data_row['Yield_Curve_Spread'] < spread_flat_thresh and
                   pd.notna(data_row['Yield_Curve_Spread_Change']) and data_row['Yield_Curve_Spread_Change'] < change_thresh)
        
        rule9_passed = cond1_yc or cond2_yc
        rule9_score = weight if rule9_passed else 0
        score += rule9_score

        if store_details:
            rule_details['rule9_yield_curve_warn'] = {
                'name': 'Yield Curve Warning',
                'description': f'Spread < {spread_abs_thresh} OR (Spread < {spread_flat_thresh} AND Change < {change_thresh})',
                'weight': weight,
                'passed': rule9_passed,
                'score': rule9_score,
                'values': {
                    'Yield_Curve_Spread': data_row['Yield_Curve_Spread'] if pd.notna(data_row['Yield_Curve_Spread']) else 0,
                    'Yield_Curve_Spread_Change': data_row['Yield_Curve_Spread_Change'] if pd.notna(data_row['Yield_Curve_Spread_Change']) else 0,
                    'Abs_Threshold': spread_abs_thresh,
                    'Flat_Threshold': spread_flat_thresh,
                    'Change_Threshold': change_thresh,
                    'Condition1_Met': cond1_yc,
                    'Condition2_Met': cond2_yc
                }
            }

        # Rule 10: Flight-to-Quality
        weight = config['rule10_flight_quality']['weight']
        weight_sum += weight
        gld_spy_ratio_thresh = config['rule10_flight_quality']['gld_spy_ratio_threshold']
        tlt_thresh = config['rule10_flight_quality']['tlt_threshold']
        
        cond1_fq = (pd.notna(data_row['GLD_SPY_Ratio']) and pd.notna(data_row['GLD_SPY_Ratio_63d_Mean']) and 
                    data_row['GLD_SPY_Ratio_63d_Mean'] > 0 and
                    data_row['GLD_SPY_Ratio'] > (data_row['GLD_SPY_Ratio_63d_Mean'] * gld_spy_ratio_thresh))
        
        cond2_fq = (pd.notna(data_row['TLT']) and pd.notna(data_row['TLT_20d_Max']) and 
                    data_row['TLT_20d_Max'] > 0 and
                    data_row['TLT'] > (data_row['TLT_20d_Max'] * tlt_thresh))
        
        rule10_passed = cond1_fq or cond2_fq
        rule10_score = weight if rule10_passed else 0
        score += rule10_score

        if store_details:
            rule_details['rule10_flight_quality'] = {
                'name': 'Flight-to-Quality',
                'description': f'GLD/SPY Ratio > 63d Mean * {gld_spy_ratio_thresh} OR TLT > 20d Max * {tlt_thresh}',
                'weight': weight,
                'passed': rule10_passed,
                'score': rule10_score,
                'values': {
                    'GLD_SPY_Ratio': data_row['GLD_SPY_Ratio'] if pd.notna(data_row['GLD_SPY_Ratio']) else 0,
                    'GLD_SPY_Ratio_63d_Mean': data_row['GLD_SPY_Ratio_63d_Mean'] if pd.notna(data_row['GLD_SPY_Ratio_63d_Mean']) else 0,
                    'TLT': data_row['TLT'] if pd.notna(data_row['TLT']) else 0,
                    'TLT_20d_Max': data_row['TLT_20d_Max'] if pd.notna(data_row['TLT_20d_Max']) else 0,
                    'GLD_Threshold': gld_spy_ratio_thresh,
                    'TLT_Threshold': tlt_thresh,
                    'Condition1_Met': cond1_fq,
                    'Condition2_Met': cond2_fq
                }
            }

    except KeyError as e:
        if store_details:
            rule_details['error'] = f"Missing key {e} in data_row for Bear scoring"
        pass
    except Exception as e:
        if store_details:
            rule_details['error'] = f"Error scoring Bear regime: {e}"
        print(f"Error scoring Bear regime for date {data_row.get('Date', 'Unknown')}: {e}")

    return min(score, 100), rule_details

# ======== MAIN CALCULATION FUNCTION ========\n
# ======== MODIFIED MAIN CALCULATION FUNCTION (WITH DEBUG PRINTS) ========

def calculate_all_regimes(df, config):
    """Calculates raw scores, applies persistence, and determines the final regime with detailed rule tracking."""
    print("Calculating regime scores...")
    n_rows = len(df)
    
    # Initialize columns
    regime_names = ['Bull', 'Neutral', 'Bear']
    for r in regime_names:
        df[f'raw_score_{r}'] = 0.0
        df[f'final_score_{r}'] = 0.0
    df['regime'] = 'Neutral' # Default
    df['confidence'] = 'Low'

    # Create a dictionary to store detailed rule information
    rule_details_by_date = {} 
    print("Initialized empty rule_details_by_date dictionary.") # DEBUG PRINT

    # --- Add helper columns needed for scoring functions ---
    # Shifted close for Volume Confirmation rule
    df['Close_Shift1'] = df['Close'].shift(1)
    # Rolling mean of Volatility for Bull Rule 4
    df['Volatility_63d_Mean'] = df['Volatility'].rolling(window=63).mean()
    # Rolling metrics for Neutral Rule 10
    df['BB_PercentB_10d_Std'] = df['BB_PercentB'].rolling(window=10).std()
    df['BB_PercentB_10d_Mean'] = df['BB_PercentB'].rolling(window=10).mean()
    # Momentum Z for Neutral Rule 4
    df['Momentum_Mean'] = df['Momentum'].rolling(window=VOL_WINDOW).mean()
    df['Momentum_Std'] = df['Momentum'].rolling(window=VOL_WINDOW).std()
    df['Momentum_Z'] = np.where(df['Momentum_Std'] != 0, (df['Momentum'] - df['Momentum_Mean']) / df['Momentum_Std'], 0)

    # --- Pre-calculate trend persistence flags ---
    # Bull Rule 1 Persistence
    bull_trend_cond = df['SMA_Fast'] > df['SMA_Slow']
    df['bull_trend_persistence'] = bull_trend_cond.rolling(window=config['Bull']['rule1_trend_structure']['persistence_days']).sum() >= config['Bull']['rule1_trend_structure']['persistence_days']

    # Bear Rule 1 Persistence
    bear_trend_cond = df['SMA_Fast'] < (df['SMA_Slow'] * config['Bear']['rule1_trend_structure']['sma_threshold'])
    df['bear_trend_persistence'] = bear_trend_cond.rolling(window=config['Bear']['rule1_trend_structure']['persistence_days']).sum() >= config['Bear']['rule1_trend_structure']['persistence_days']
    
    # --- Iterate and Score ---
    print(f"Starting loop for {n_rows - 2} dates (from index 2 to {n_rows - 1})...") # DEBUG PRINT
    
    for i in range(2, n_rows):
        current_row = df.iloc[i]
        # Ensure the index is a Timestamp, handle potential MultiIndex if not set correctly before
        if isinstance(df.index, pd.MultiIndex):
             date = df.index[i][0] # Assuming Date is the first level
        else:
             date = df.index[i] 
             
        # Ensure date is a Timestamp for dictionary keys
        if not isinstance(date, pd.Timestamp):
            date = pd.Timestamp(date)

        # Calculate raw scores with detailed rule tracking
        bull_raw_score, bull_details = score_bull_regime(current_row, config['Bull'])
        neutral_raw_score, neutral_details = score_neutral_regime(current_row, config['Neutral'])
        bear_raw_score, bear_details = score_bear_regime(current_row, config['Bear'])
        
        raw_scores = {
            'Bull': bull_raw_score,
            'Neutral': neutral_raw_score,
            'Bear': bear_raw_score
        }

        # --- Apply Persistence Rule Corrections ---
        # Check pre-calculated persistence flags and adjust score if needed
        # Bull Rule 1
        if not current_row['bull_trend_persistence'] and (pd.notna(current_row['SMA_Fast']) and pd.notna(current_row['SMA_Slow']) and current_row['SMA_Fast'] > current_row['SMA_Slow']):
             raw_scores['Bull'] -= config['Bull']['rule1_trend_structure']['weight']
             if bull_details and 'rule1_trend_structure' in bull_details:
                 bull_details['rule1_trend_structure']['score'] = 0
                 bull_details['rule1_trend_structure']['passed'] = False
                 bull_details['rule1_trend_structure']['persistence_failed'] = True
                 
        # Bear Rule 1
        if not current_row['bear_trend_persistence'] and (pd.notna(current_row['SMA_Fast']) and pd.notna(current_row['SMA_Slow']) and current_row['SMA_Slow'] > 0 and current_row['SMA_Fast'] < (current_row['SMA_Slow'] * config['Bear']['rule1_trend_structure']['sma_threshold'])):
             raw_scores['Bear'] -= config['Bear']['rule1_trend_structure']['weight']
             if bear_details and 'rule1_trend_structure' in bear_details:
                 bear_details['rule1_trend_structure']['score'] = 0
                 bear_details['rule1_trend_structure']['passed'] = False
                 bear_details['rule1_trend_structure']['persistence_failed'] = True
             
        # Ensure scores are not negative after correction
        raw_scores['Bull'] = max(0, raw_scores['Bull'])
        raw_scores['Bear'] = max(0, raw_scores['Bear'])

        # Store raw scores in DataFrame using .loc with the date index
        for r in regime_names:
            df.loc[date, f'raw_score_{r}'] = raw_scores[r]

        # Apply Persistence Bonus
        final_scores = raw_scores.copy()
        
        # Previous regimes for persistence (use .loc with previous dates)
        prev_date_1 = df.index[i-1]
        prev_date_2 = df.index[i-2]
        prev_regime = df.loc[prev_date_1, 'regime']
        prev_regime_2 = df.loc[prev_date_2, 'regime']
        persistence_cfg = config['Persistence']
        
        # Track persistence bonuses for the scorecard
        persistence_bonuses = {r: 0 for r in regime_names}

        if prev_regime in final_scores:
            final_scores[prev_regime] += persistence_cfg['t-1']
            persistence_bonuses[prev_regime] += persistence_cfg['t-1']
            
        if prev_regime_2 in final_scores:
             final_scores[prev_regime_2] += persistence_cfg['t-2']
             persistence_bonuses[prev_regime_2] += persistence_cfg['t-2']

        # Store final scores in DataFrame using .loc
        for r in regime_names:
            df.loc[date, f'final_score_{r}'] = final_scores[r]

        # Determine winning regime with tiebreaker: Bear > Neutral > Bull
        max_score = -1
        winning_regime = 'Neutral' # Default

        # Evaluate in order of tiebreaker priority
        if final_scores['Bear'] > max_score:
            max_score = final_scores['Bear']
            winning_regime = 'Bear'
        if final_scores['Neutral'] >= max_score: # Neutral wins ties vs Bull
            if winning_regime != 'Bear' or final_scores['Neutral'] > max_score: # Only override Bear if strictly greater
                max_score = final_scores['Neutral']
                winning_regime = 'Neutral'
        if final_scores['Bull'] > max_score: # Bull must strictly beat others
            max_score = final_scores['Bull']
            winning_regime = 'Bull'

        # Store winning regime in DataFrame using .loc
        df.loc[date, 'regime'] = winning_regime

        # Calculate Confidence
        sorted_scores = sorted(final_scores.values(), reverse=True)
        score_margin = sorted_scores[0] - sorted_scores[1] if len(sorted_scores) > 1 else sorted_scores[0]
        if score_margin > 25:
            confidence = "High"
        elif score_margin > 15:
            confidence = "Medium"
        else:
            confidence = "Low"
            
        # Store confidence in DataFrame using .loc
        df.loc[date, 'confidence'] = confidence
        
        # --- Store all details for this date ---
        current_details_for_date = {
            'raw_scores': raw_scores,
            'final_scores': final_scores,
            'persistence_bonuses': persistence_bonuses,
            'winning_regime': winning_regime,
            'confidence': confidence,
            'score_margin': score_margin,
            'Bull': bull_details,
            'Neutral': neutral_details,
            'Bear': bear_details
        }
        
        rule_details_by_date[date] = current_details_for_date # Assign details for the current date

        # --- DEBUG PRINTS ---
        if i % 50 == 2 or i == n_rows - 1: # Print near start, every 50 iterations, and the last one
             log_date_str = date.strftime('%Y-%m-%d') if isinstance(date, pd.Timestamp) else str(date)
             print(f"  -> Stored details for {log_date_str}. Current dictionary size: {len(rule_details_by_date)}")
             if len(rule_details_by_date) > 1:
                 first_key = next(iter(rule_details_by_date))
                 first_key_str = first_key.strftime('%Y-%m-%d') if isinstance(first_key, pd.Timestamp) else str(first_key)
                 print(f"     First key remains: {first_key_str}")
        # --- END DEBUG PRINTS ---

    print("\nLoop finished.") # DEBUG PRINT
    print(f"Final size of rule_details_by_date before assigning: {len(rule_details_by_date)}") # DEBUG PRINT
    if rule_details_by_date:
        first_key_final = next(iter(rule_details_by_date))
        last_key_final = list(rule_details_by_date.keys())[-1]
        first_key_final_str = first_key_final.strftime('%Y-%m-%d') if isinstance(first_key_final, pd.Timestamp) else str(first_key_final)
        last_key_final_str = last_key_final.strftime('%Y-%m-%d') if isinstance(last_key_final, pd.Timestamp) else str(last_key_final)
        print(f"First date key: {first_key_final_str}") # DEBUG PRINT
        print(f"Last date key: {last_key_final_str}") # DEBUG PRINT
    # --- END DEBUG PRINTS ---
    
    # Store all the rule details as an attribute of the dataframe
    # This attribute is not a standard pandas feature, it's a custom addition
    df.rule_details = rule_details_by_date 
    print(f"Size of df.rule_details after assignment: {len(df.rule_details) if hasattr(df, 'rule_details') else 'Attribute not found!'}") # DEBUG PRINT
    
    # Clean up helper columns added within this function if desired (optional)
    # cols_to_drop = ['Close_Shift1', 'Volatility_63d_Mean', 'BB_PercentB_10d_Std', ...] 
    # df = df.drop(columns=cols_to_drop, errors='ignore')
    
    print("Regime scoring calculation function complete.") # DEBUG PRINT
    return df

# ======== ANALYSIS & VISUALIZATION FUNCTIONS (Adapted) ========\n
def calculate_regime_statistics(df):
    """Calculate key statistics for each market regime - adapted for rule-based output"""
    print("\nCalculating statistics per regime...")
    # Ensure 'regime' column exists
    if 'regime' not in df.columns:
        print("Error: 'regime' column not found in DataFrame. Cannot calculate stats.")
        return pd.DataFrame()

    regime_stats = {}
    # Use the actual regime labels present in the column
    unique_labels = df['regime'].unique()
    # Filter out potential initial NaNs or default values if loop starts later
    unique_labels = [label for label in unique_labels if pd.notna(label) and label in ['Bull', 'Neutral', 'Bear']]


    for label in unique_labels:
        mask = (df['regime'] == label)
        count = mask.sum()

        if count == 0:
            print(f"Warning: No data points found for label '{label}'. Skipping stats calculation.")
            continue

        regime_data = df[mask].copy()
        returns = regime_data['Log_Return'] # Use log returns calculated earlier

        # Basic stats
        return_mean = returns.mean()
        return_std = returns.std()
        ann_return = return_mean * 252
        ann_vol = return_std * np.sqrt(252)
        sharpe = ann_return / ann_vol if ann_vol > 0 else 0
        hit_rate = (returns > 0).mean() * 100

        # Avg values of key indicators
        avg_vix = regime_data['VIX'].mean() if 'VIX' in regime_data.columns else np.nan
        avg_vol = regime_data['Volatility'].mean() if 'Volatility' in regime_data.columns else np.nan
        avg_mom = regime_data['Momentum'].mean() if 'Momentum' in regime_data.columns else np.nan
        avg_chop = regime_data['Choppiness_Index'].mean() if 'Choppiness_Index' in regime_data.columns else np.nan
        avg_hyg_tlt_z = regime_data['HYG_TLT_Z'].mean() if 'HYG_TLT_Z' in regime_data.columns else np.nan


        # Max Drawdown calculation within the regime period
        if len(regime_data) > 1:
             cumulative_ret = (1 + returns / 100).cumprod()
             peak = cumulative_ret.expanding(min_periods=1).max()
             drawdown = (cumulative_ret / peak - 1) * 100
             max_drawdown = drawdown.min()
        else:
             max_drawdown = 0 # Or NaN

        regime_stats[label] = {
            'count': count,
            'return_mean_daily': return_mean,
            'return_std_daily': return_std,
            'annualized_return': ann_return,
            'annualized_volatility': ann_vol,
            'sharpe_ratio': sharpe,
            'hit_rate_pct': hit_rate,
            'max_drawdown_pct': max_drawdown,
            'avg_vix': avg_vix,
            'avg_realized_vol': avg_vol,
            'avg_momentum_63d': avg_mom,
            'avg_choppiness': avg_chop,
            'avg_credit_z': avg_hyg_tlt_z,
        }

    stats_df = pd.DataFrame.from_dict(regime_stats, orient='index')
    print("Regime statistics calculated.")
    return stats_df


def analyze_regime_transitions(df):
    """Analyze transitions between regimes"""
    print("\nAnalyzing regime transitions...")
    if 'regime' not in df.columns or df['regime'].isnull().all():
         print("Error: 'regime' column missing or empty. Cannot analyze transitions.")
         return [], pd.DataFrame()
         
    # Ensure Date is index or column
    if isinstance(df.index, pd.DatetimeIndex):
        dates = df.index
    elif 'Date' in df.columns:
         dates = df['Date']
    else:
         print("Error: Cannot find Date information.")
         return [], pd.DataFrame()


    transitions = []
    df_shifted = df['regime'].shift(1)
    changes = df[df['regime'] != df_shifted]

    if changes.empty:
        print("No regime transitions detected.")
        # Create a dummy transition representing the single regime found
        start_date = dates[0] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[0]
        end_date = dates[-1] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[-1]
        duration = (end_date - start_date).days
        current_regime = df['regime'].iloc[0]
        transitions.append({
            'from_regime': None, # No prior regime
            'to_regime': current_regime,
            'start_date': start_date,
            'end_date': end_date,
            'duration_days': duration
        })
        return transitions, pd.DataFrame() # Return empty transition matrix

    last_regime = df['regime'].iloc[0]
    last_date = dates[0] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[0]

    # Add the first period
    first_change_date = changes.index[0] if isinstance(changes.index, pd.DatetimeIndex) else changes['Date'].iloc[0]
    first_change_idx = df.index.get_loc(first_change_date) if isinstance(df.index, pd.DatetimeIndex) else dates[dates == first_change_date].index[0]
    
    start_date = dates[0] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[0]
    end_date = dates[first_change_idx - 1] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[first_change_idx - 1]
    duration = (end_date - start_date).days
    transitions.append({
        'from_regime': None,
        'to_regime': last_regime,
        'start_date': start_date,
        'end_date': end_date,
        'duration_days': duration
    })


    for i in range(len(changes)):
        current_change_date = changes.index[i] if isinstance(changes.index, pd.DatetimeIndex) else changes['Date'].iloc[i]
        current_regime = changes['regime'].iloc[i]
        
        start_date = current_change_date
        # Find end date (day before next change, or last day)
        if i + 1 < len(changes):
             next_change_date = changes.index[i+1] if isinstance(changes.index, pd.DatetimeIndex) else changes['Date'].iloc[i+1]
             next_change_idx = df.index.get_loc(next_change_date) if isinstance(df.index, pd.DatetimeIndex) else dates[dates == next_change_date].index[0]
             end_date = dates[next_change_idx - 1] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[next_change_idx - 1]
        else:
             end_date = dates[-1] if isinstance(dates, pd.DatetimeIndex) else dates.iloc[-1]

        duration = (end_date - start_date).days + 1 # Add 1 day for duration calculation

        transitions.append({
            'from_regime': last_regime,
            'to_regime': current_regime,
            'start_date': start_date,
            'end_date': end_date,
            'duration_days': duration
        })
        last_regime = current_regime


    # --- Create Transition Matrix ---
    from_regimes = [t['from_regime'] for t in transitions if t['from_regime'] is not None]
    to_regimes = [t['to_regime'] for t in transitions if t['from_regime'] is not None]
    
    labels = sorted(list(set(from_regimes + to_regimes)))
    matrix = pd.DataFrame(0, index=labels, columns=labels)

    for from_r, to_r in zip(from_regimes, to_regimes):
        matrix.loc[from_r, to_r] += 1

    # Normalize to probabilities
    matrix_prob = matrix.div(matrix.sum(axis=1), axis=0).fillna(0)

    print("Regime transition analysis complete.")
    return transitions, matrix_prob


def plot_regime_time_series(df):
    """Plot the market price with regime classifications over time"""
    if 'regime' not in df.columns or 'Close' not in df.columns:
        print("Error: Missing 'regime' or 'Close' column for plotting.")
        return
        
    # Ensure Date is index or column
    if isinstance(df.index, pd.DatetimeIndex):
        date_col = df.index
    elif 'Date' in df.columns:
         date_col = df['Date']
    else:
         print("Error: Cannot find Date information for plotting.")
         return

    # Define colors for regimes
    regime_colors = {'Bull': 'rgba(0, 128, 0, 0.3)', 'Neutral': 'rgba(255, 215, 0, 0.3)', 'Bear': 'rgba(255, 0, 0, 0.3)'}
    line_colors = {'Bull': 'green', 'Neutral': 'orange', 'Bear': 'red'}
    
    fig = make_subplots(rows=3, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        subplot_titles=('Market Price with Regime Background', 'Regime Scores', 'Regime Confidence'),
                        row_heights=[0.6, 0.2, 0.2])

    # Plot 1: Price with Regime Background
    fig.add_trace(go.Scatter(x=date_col, y=df['Close'], mode='lines', name='Price', line=dict(color='black', width=1)), row=1, col=1)

    # Add shapes for regime backgrounds
    start_date = date_col[0] if isinstance(date_col, pd.DatetimeIndex) else date_col.iloc[0]
    current_regime = df['regime'].iloc[0]
    for i in range(1, len(df)):
        if df['regime'].iloc[i] != current_regime or i == len(df) - 1:
            end_date = date_col[i] if isinstance(date_col, pd.DatetimeIndex) else date_col.iloc[i]
            if i != len(df) - 1:
                end_date = date_col[i-1] if isinstance(date_col, pd.DatetimeIndex) else date_col.iloc[i-1]
            color = regime_colors.get(current_regime, 'rgba(128, 128, 128, 0.3)') # Default grey
            fig.add_shape(type="rect",
                          x0=start_date, y0=0, x1=end_date, y1=df['Close'].max()*1.1, # Adjust y1 if needed
                          line=dict(width=0), fillcolor=color, layer='below',
                          row=1, col=1)
            start_date = date_col[i] if isinstance(date_col, pd.DatetimeIndex) else date_col.iloc[i]
            current_regime = df['regime'].iloc[i]


    # Plot 2: Regime Scores
    for r in ['Bull', 'Neutral', 'Bear']:
        if f'final_score_{r}' in df.columns:
            fig.add_trace(go.Scatter(x=date_col, y=df[f'final_score_{r}'], mode='lines', name=f'{r} Score', line=dict(color=line_colors.get(r))), row=2, col=1)

    # Plot 3: Confidence
    if 'confidence' in df.columns:
         # Map confidence to numerical value for plotting
         confidence_map = {'High': 3, 'Medium': 2, 'Low': 1}
         y_confidence = df['confidence'].map(confidence_map)
         fig.add_trace(go.Scatter(x=date_col, y=y_confidence, mode='lines', name='Confidence', line=dict(color='purple')), row=3, col=1)
         fig.update_yaxes(tickvals=[1, 2, 3], ticktext=['Low', 'Medium', 'High'], range=[0.5, 3.5], row=3, col=1)


    # Update layout
    fig.update_layout(title='Market Regime Classification',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Regimes/Scores',
                      height=900,
                      template='plotly_white',
                      xaxis_rangeslider_visible=False)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    fig.update_yaxes(title_text="Confidence", row=3, col=1)
    
    fig.show()


def plot_regime_performance(stats_df):
    """Plot performance metrics for each regime"""
    if stats_df.empty:
        print("Statistics DataFrame is empty. Cannot plot performance.")
        return

    stats_df = stats_df.reset_index().rename(columns={'index': 'Regime'})
    regime_colors = {'Bull': 'green', 'Neutral': 'gold', 'Bear': 'red'}
    
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Annualized Return (%)', 'Annualized Volatility (%)', 'Sharpe Ratio', 'Max Drawdown (%)'),
                        vertical_spacing=0.15,
                        horizontal_spacing=0.1)

    # Plot 1: Ann Return
    fig.add_trace(go.Bar(x=stats_df['Regime'], y=stats_df['annualized_return'], name='Ann. Return', marker_color=[regime_colors.get(r, 'grey') for r in stats_df['Regime']]), row=1, col=1)
    # Plot 2: Ann Volatility
    fig.add_trace(go.Bar(x=stats_df['Regime'], y=stats_df['annualized_volatility'], name='Ann. Vol', marker_color=[regime_colors.get(r, 'grey') for r in stats_df['Regime']]), row=1, col=2)
    # Plot 3: Sharpe Ratio
    fig.add_trace(go.Bar(x=stats_df['Regime'], y=stats_df['sharpe_ratio'], name='Sharpe', marker_color=[regime_colors.get(r, 'grey') for r in stats_df['Regime']]), row=2, col=1)
    # Plot 4: Max Drawdown
    fig.add_trace(go.Bar(x=stats_df['Regime'], y=stats_df['max_drawdown_pct'], name='Max Drawdown', marker_color=[regime_colors.get(r, 'grey') for r in stats_df['Regime']]), row=2, col=2)

    fig.update_layout(title='Performance by Market Regime',
                      height=700,
                      template='plotly_white',
                      showlegend=False)
    fig.show()

# ======== MAIN FUNCTION ========\n
# Global variable to store analysis results for dashboard access
regime_analysis_results = None

def main():
    """Main execution function for the rule-based market regime analysis"""
    
    # 1. Download and prepare market data
    df_market = download_market_data(
        TICKER, VIX_TICKER, TNX_TICKER, GLD_TICKER,
        XLY_TICKER, XLP_TICKER, XLU_TICKER, XLF_TICKER,
        HYG_TICKER, TLT_TICKER, VIX3M_TICKER, IRX_TICKER,
        UUP_TICKER, TIP_TICKER, IEF_TICKER,
        START_DATE, END_DATE
    )

    # 2. Load A/D line data (optional)
    ad_data = load_ad_line_data("nyse_breadth_2023.csv") # Update path if needed

    # 3. Calculate all features (enhanced function)
    # Pass df_market (base market data) and ad_data
    df_features = calculate_features(df_market, ad_data, percentile_lookback=PERCENTILE_LOOKBACK)

    # Ensure we have enough data after feature calculation and NaN drops
    if len(df_features) < max(PERCENTILE_LOOKBACK, SMA_SLOW, MOMENTUM_WINDOW) + 5: # Need enough history
         print(f"Error: Insufficient data ({len(df_features)} rows) after feature calculation. Minimum needed ~{max(PERCENTILE_LOOKBACK, SMA_SLOW, MOMENTUM_WINDOW)}. Exiting.")
         return None

    # Make sure Date is the index for easier .loc operations
    if 'Date' in df_features.columns:
         df_features['Date'] = pd.to_datetime(df_features['Date'])
         df_features = df_features.set_index('Date')


    # 4. Calculate regimes using the rule-based system
    df_final = calculate_all_regimes(df_features, ALL_CONFIG)

    # --- Verification Steps ---
    print(f"\nFinal DataFrame head:\n{df_final.head().to_string()}")
    print(f"\nFinal DataFrame tail:\n{df_final.tail().to_string()}")
    print(f"\nRegime distribution:\n{df_final['regime'].value_counts(normalize=True) * 100}")


    # 5. Calculate regime statistics
    regime_stats_df = calculate_regime_statistics(df_final)
    if not regime_stats_df.empty:
        print("\nRegime Performance Summary:")
        print("=" * 80)
        # Format for better readability
        print(regime_stats_df.to_string(float_format="%.2f"))
    else:
         print("\nRegime statistics could not be calculated.")


    # 6. Analyze regime transitions
    transitions_list, transition_matrix = analyze_regime_transitions(df_final)
    if transitions_list:
        print("\nRegime Transitions Summary:")
        print("=" * 80)
        # Print last 10 transitions
        for t in transitions_list[-10:]:
             from_r = t['from_regime'] if t['from_regime'] else "Start"
             print(f"{from_r} → {t['to_regime']}: {t['start_date'].strftime('%Y-%m-%d')} to {t['end_date'].strftime('%Y-%m-%d')} ({t['duration_days']} days)")
        
        print("\nTransition Matrix (Probability):")
        print("=" * 80)
        print(transition_matrix.to_string(float_format="%.3f"))


    # 7. Create visualizations
    plot_regime_time_series(df_final)
    if not regime_stats_df.empty:
        plot_regime_performance(regime_stats_df)


    print("\nAnalysis Complete.")
    # Return the final dataframe for further external use if needed
    # Return the final dataframe and other results
    return df_final, regime_stats_df, transitions_list, transition_matrix
   

if __name__ == "__main__":
    results_df, stats, transitions, matrix = main()
    
    # Store results in global variable for dashboard access
    global regime_analysis_results
    regime_analysis_results = {
        'data': results_df,
        'stats': stats,
        'transitions': transitions,
        'transition_matrix': matrix,
        'config': ALL_CONFIG,
        'last_update': datetime.now()
    }
    
    if results_df is not None:
        print("\nRule-based regime analysis finished successfully.")
        latest_regime = results_df['regime'].iloc[-1]
        latest_confidence = results_df['confidence'].iloc[-1]
        latest_scores = results_df[['final_score_Bull', 'final_score_Neutral', 'final_score_Bear']].iloc[-1]
        print(f"\nLatest Calculated Regime ({results_df.index[-1].strftime('%Y-%m-%d')}): {latest_regime} (Confidence: {latest_confidence})")
        print(f"Scores: Bull={latest_scores['final_score_Bull']:.1f}, Neutral={latest_scores['final_score_Neutral']:.1f}, Bear={latest_scores['final_score_Bear']:.1f}")
    else:
        print("\nAnalysis failed or returned no results.")



Merging dataframes...
Forward filling missing values...
Market data download and initial preparation complete.
Loading A/D line data from nyse_breadth_2023.csv...
A/D line data loaded and prepared.
Calculating technical features...
Calculating rolling percentiles...
Merging A/D line features...
Dropped 63 rows due to NaNs in essential features.
Feature calculation complete.
Calculating regime scores...
Initialized empty rule_details_by_date dictionary.
Starting loop for 1774 dates (from index 2 to 1775)...
  -> Stored details for 2018-04-06. Current dictionary size: 1
  -> Stored details for 2018-06-18. Current dictionary size: 51
     First key remains: 2018-04-06
  -> Stored details for 2018-08-28. Current dictionary size: 101
     First key remains: 2018-04-06
  -> Stored details for 2018-11-07. Current dictionary size: 151
     First key remains: 2018-04-06
  -> Stored details for 2019-01-23. Current dictionary size: 201
     First key remains: 2018-04-06
  -> Stored details for 20


Analysis Complete.

Rule-based regime analysis finished successfully.

Latest Calculated Regime (2025-04-25): Bear (Confidence: Medium)
Scores: Bull=11.0, Neutral=29.5, Bear=47.2


In [6]:
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML

# --- Verification Cell ---

def verify_detailed_data(results=regime_analysis_results, verification_date=None):
    """
    Verifies that the detailed scoring data is present for a given date.
    """
    if results is None:
        print("❌ Error: 'regime_analysis_results' not found. Run the main analysis first.")
        return

    if not hasattr(results['data'], 'rule_details') or not isinstance(results['data'].rule_details, dict):
        print("❌ Error: '.rule_details' attribute not found or is not a dictionary in the results DataFrame.")
        print("   Ensure the modified 'calculate_all_regimes' function was used and ran successfully.")
        return

    all_details = results['data'].rule_details
    available_dates = sorted(list(all_details.keys()))

    if not available_dates:
        print("❌ Error: No dates found in '.rule_details'. The calculation might have failed.")
        return

    # Use the last available date if none is specified
    if verification_date is None:
        verification_date = available_dates[-1]
    elif isinstance(verification_date, str):
        # Attempt to parse string date
        try:
            verification_date = pd.to_datetime(verification_date).normalize()
        except ValueError:
            print(f"❌ Error: Invalid date string format '{verification_date}'. Use YYYY-MM-DD.")
            return
            
    # Ensure the date is a Timestamp (like the keys in rule_details)
    if not isinstance(verification_date, pd.Timestamp):
         verification_date = pd.Timestamp(verification_date).normalize()


    print(f"--- Verifying Data for Date: {verification_date.strftime('%Y-%m-%d')} ---")

    if verification_date not in all_details:
        print(f"❌ Error: Date {verification_date.strftime('%Y-%m-%d')} not found in the detailed results.")
        print(f"   Available dates range from {available_dates[0].strftime('%Y-%m-%d')} to {available_dates[-1].strftime('%Y-%m-%d')}.")
        return

    date_details = all_details[verification_date]

    # Check top-level structure
    expected_keys = ['raw_scores', 'final_scores', 'persistence_bonuses', 'winning_regime', 'confidence', 'score_margin', 'Bull', 'Neutral', 'Bear']
    missing_keys = [key for key in expected_keys if key not in date_details]

    if missing_keys:
        print(f"❌ Error: Missing top-level keys for the selected date: {', '.join(missing_keys)}")
        return

    print("✅ Top-level structure looks correct.")

    # Print Summary Information
    print("\n--- Summary Scores ---")
    print(f"Winning Regime: {date_details['winning_regime']}")
    print(f"Confidence: {date_details['confidence']}")
    print(f"Score Margin: {date_details['score_margin']:.2f}")
    
    scores_df = pd.DataFrame({
        'Regime': ['Bull', 'Neutral', 'Bear'],
        'Raw Score': [date_details['raw_scores']['Bull'], date_details['raw_scores']['Neutral'], date_details['raw_scores']['Bear']],
        'Persistence Bonus': [date_details['persistence_bonuses']['Bull'], date_details['persistence_bonuses']['Neutral'], date_details['persistence_bonuses']['Bear']],
        'Final Score': [date_details['final_scores']['Bull'], date_details['final_scores']['Neutral'], date_details['final_scores']['Bear']]
    })
    display(scores_df)

    # Check Structure of Regime Details (e.g., Bull)
    print("\n--- Detailed Rule Structure Check (Using Bull Regime) ---")
    bull_details = date_details['Bull']
    if not isinstance(bull_details, dict) or not bull_details:
        print("❌ Error: Bull regime details are missing or not a dictionary.")
        return

    # Get the first rule detail entry
    first_rule_id = next(iter(bull_details))
    first_rule_detail = bull_details[first_rule_id]

    print(f"Checking structure of the first Bull rule ('{first_rule_id}')...")
    expected_rule_keys = ['name', 'description', 'weight', 'passed', 'score', 'values']
    missing_rule_keys = [key for key in expected_rule_keys if key not in first_rule_detail]

    if missing_rule_keys:
        print(f"❌ Error: Missing keys in rule detail structure: {', '.join(missing_rule_keys)}")
        return

    print("✅ Rule detail structure looks correct.")

    # Print a sample rule detail
    print("\n--- Sample Rule Detail (Bull Regime) ---")
    print(f"Rule ID: {first_rule_id}")
    for key, value in first_rule_detail.items():
        if key == 'values':
            print(f"  {key}:")
            for v_key, v_value in value.items():
                print(f"    {v_key}: {v_value}")
        else:
            print(f"  {key}: {value}")

    print("\n--- Verification Complete ---")
    print("If all checks passed (✅), the detailed data structure is likely correct.")

# --- How to Use ---
# 1. Run the main analysis cell first.
# 2. Run this cell. By default, it checks the *last* date in the analysis.
# 3. To check a specific date, pass it as an argument:
verify_detailed_data(verification_date='2024-04-24') # Use YYYY-MM-DD format
verify_detailed_data(verification_date='2024-04-25') # Use YYYY-MM-DD format
#    verify_detailed_data(verification_date=pd.Timestamp('2023-11-30'))

# Run the verification for the latest date
#verify_detailed_data() 


--- Verifying Data for Date: 2024-04-24 ---
✅ Top-level structure looks correct.

--- Summary Scores ---
Winning Regime: Bear
Confidence: Low
Score Margin: 14.29


,Regime,Raw Score,Persistence Bonus,Final Score
0,Bull,24.00,0,24.00
1,Neutral,14.52,0,14.52
2,Bear,26.29,12,38.29



--- Detailed Rule Structure Check (Using Bull Regime) ---
Checking structure of the first Bull rule ('rule1_trend_structure')...
✅ Rule detail structure looks correct.

--- Sample Rule Detail (Bull Regime) ---
Rule ID: rule1_trend_structure
  name: Trend Structure
  description: SMA Fast > SMA Slow (persistence checked separately)
  weight: 12
  passed: True
  score: 12
  values:
    SMA_Fast: 505.22721099853516
    SMA_Slow: 503.54705688476565
    Persistence: True

--- Verification Complete ---
If all checks passed (✅), the detailed data structure is likely correct.
--- Verifying Data for Date: 2024-04-25 ---
✅ Top-level structure looks correct.

--- Summary Scores ---
Winning Regime: Bear
Confidence: High
Score Margin: 29.25


,Regime,Raw Score,Persistence Bonus,Final Score
0,Bull,24.00,0,24.00
1,Neutral,14.52,0,14.52
2,Bear,41.25,12,53.25



--- Detailed Rule Structure Check (Using Bull Regime) ---
Checking structure of the first Bull rule ('rule1_trend_structure')...
✅ Rule detail structure looks correct.

--- Sample Rule Detail (Bull Regime) ---
Rule ID: rule1_trend_structure
  name: Trend Structure
  description: SMA Fast > SMA Slow (persistence checked separately)
  weight: 12
  passed: True
  score: 12
  values:
    SMA_Fast: 504.2555786132813
    SMA_Slow: 503.76311462402344
    Persistence: True

--- Verification Complete ---
If all checks passed (✅), the detailed data structure is likely correct.


In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

def create_market_regime_dashboard(results=regime_analysis_results):
    """Create interactive dashboard for market regime analysis with pyramid exploration"""
    if results is None or not hasattr(results['data'], 'rule_details'):
        print("Error: Run the main analysis first or rule details not available")
        return
    
    # Extract data
    df = results['data']
    rule_details = df.rule_details
    config = results['config']
    
    # Create date picker
    available_dates = sorted(list(rule_details.keys()))
    
    date_picker = widgets.DatePicker(
        description='Select date:',
        disabled=False,
        value=available_dates[-1]
    )
    
    # Create regime selector
    regime_selector = widgets.Dropdown(
        options=['Summary', 'Bull', 'Neutral', 'Bear'],
        value='Summary',
        description='View:',
        disabled=False
    )
    
    # Create dashboard output
    dashboard_output = widgets.Output()
    
    # Create exploration level indicators
    exploration_level = widgets.IntProgress(
        value=1,
        min=1,
        max=4,
        step=1,
        description='Drill Down:',
        bar_style='info',
        orientation='horizontal'
    )
    
    level_descriptions = widgets.HTML(
        value='<div style="font-size:12px;margin-top:-15px;margin-bottom:10px"><span style="color:#9e9e9e">Level 1: Overview → Level 2: Regime Scores → Level 3: Rule Groups → Level 4: Rule Details</span></div>'
    )
    
    # Store the current exploration state
    state = {
        'date': available_dates[-1],
        'regime': 'Summary',
        'level': 1,
        'selected_rule': None
    }
    
    # Define color schemes
    colors = {
        'Bull': {'primary': 'green', 'light': 'rgba(0, 128, 0, 0.2)', 'medium': 'rgba(0, 128, 0, 0.5)'},
        'Neutral': {'primary': 'orange', 'light': 'rgba(255, 165, 0, 0.2)', 'medium': 'rgba(255, 165, 0, 0.5)'},
        'Bear': {'primary': 'red', 'light': 'rgba(255, 0, 0, 0.2)', 'medium': 'rgba(255, 0, 0, 0.5)'}
    }
    
    def update_dashboard():
        """Update dashboard based on current state"""
        with dashboard_output:
            clear_output()
            
            # Get data for selected date
            if state['date'] not in rule_details:
                display(HTML(f"<h3>No data available for {state['date']}</h3>"))
                return
                
            date_details = rule_details[state['date']]
            row = df.loc[state['date']]
            
            # Level 1: Overview (Main chart + regime summary)
            if state['level'] == 1:
                show_level1_overview(state['date'], row, date_details)
            
            # Level 2: Regime Scores Breakdown 
            elif state['level'] == 2:
                if state['regime'] == 'Summary':
                    show_level2_regime_comparison(state['date'], row, date_details)
                else:
                    show_level2_regime_detail(state['date'], state['regime'], row, date_details)
            
            # Level 3: Rule Group Analysis
            elif state['level'] == 3:
                if state['regime'] != 'Summary':
                    show_level3_rule_breakdown(state['date'], state['regime'], date_details)
                else:
                    # Fallback to level 2 if summary is selected
                    show_level2_regime_comparison(state['date'], row, date_details)
                    
            # Level 4: Individual Rule Details
            elif state['level'] == 4:
                if state['regime'] != 'Summary' and state['selected_rule'] is not None:
                    show_level4_rule_detail(state['date'], state['regime'], 
                                          state['selected_rule'], date_details)
                else:
                    # Fallback to level 3
                    show_level3_rule_breakdown(state['date'], state['regime'], date_details)
    
    def show_level1_overview(date, row, details):
        """Show top-level overview with price chart and regime"""
        # Create header
        regime = details['winning_regime']
        confidence = details['confidence']
        
        header_html = f"""
        <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px;">
            <h2>Market Regime Dashboard: {date.strftime('%Y-%m-%d')}</h2>
            <h3>Current Regime: <span style="color:{colors[regime]['primary']}">{regime}</span> 
            (Confidence: {confidence})</h3>
        </div>
        """
        display(HTML(header_html))
        
        # Create the price chart with regime background
        # Look back about 6 months
        start_date = date - timedelta(days=180)
        end_date = date
        
        if start_date < df.index[0]:
            start_date = df.index[0]
        
        # Subset the data
        mask = (df.index >= start_date) & (df.index <= end_date)
        chart_df = df[mask].copy()
        
        # Create figure with subplots
        fig = make_subplots(rows=3, cols=1,
                           shared_xaxes=True,
                           vertical_spacing=0.03,
                           subplot_titles=('Price with Regime Background', 'Regime Scores', 'Market Indicators'),
                           row_heights=[0.5, 0.25, 0.25])
        
        # Add price chart
        fig.add_trace(go.Scatter(
            x=chart_df.index,
            y=chart_df['Close'],
            mode='lines',
            name='Price',
            line=dict(color='black', width=2)
        ), row=1, col=1)
        
        # Add regime backgrounds
        regime_changes = chart_df['regime'].ne(chart_df['regime'].shift()).cumsum()
        for i, (_, group) in enumerate(chart_df.groupby(regime_changes)):
            if not group.empty:
                current_regime = group['regime'].iloc[0]
                start = group.index[0]
                end = group.index[-1]
                
                fig.add_shape(
                    type="rect",
                    x0=start, y0=chart_df['Close'].min() * 0.95,
                    x1=end, y1=chart_df['Close'].max() * 1.05,
                    fillcolor=colors[current_regime]['light'],
                    line=dict(width=0),
                    layer="below",
                    row=1, col=1
                )
        
        # Highlight the selected date
        fig.add_shape(
            type="line",
            x0=date, y0=chart_df['Close'].min() * 0.95,
            x1=date, y1=chart_df['Close'].max() * 1.05,
            line=dict(color="black", width=2, dash="dash"),
            row=1, col=1
        )
        
        # Add regime scores
        for regime_name in ['Bull', 'Neutral', 'Bear']:
            fig.add_trace(go.Scatter(
                x=chart_df.index,
                y=chart_df[f'final_score_{regime_name}'],
                mode='lines',
                name=f'{regime_name} Score',
                line=dict(color=colors[regime_name]['primary'])
            ), row=2, col=1)
        
        # Add some key indicators
        # Volatility
        if 'Volatility' in chart_df.columns:
            fig.add_trace(go.Scatter(
                x=chart_df.index,
                y=chart_df['Volatility'],
                mode='lines',
                name='Volatility',
                line=dict(color='purple')
            ), row=3, col=1)
        
        # VIX
        if 'VIX' in chart_df.columns:
            fig.add_trace(go.Scatter(
                x=chart_df.index,
                y=chart_df['VIX'],
                mode='lines',
                name='VIX',
                line=dict(color='orange')
            ), row=3, col=1)
        
        fig.update_layout(
            height=800,
            title_text=f"Market Regime Analysis - Click to Drill Down",
            hovermode="x unified",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
        )
        
        # Enable click to drill down
        fig.update_layout(clickmode='event')
        
        # Display interactive features message
        display(HTML("""
        <div style="text-align:center; margin-top:-15px; margin-bottom:15px; color:#666;">
            <em>Click chart to drill down or use the controls above</em>
        </div>
        """))
        
        # Display the figure
        display(fig)
        
        # Add key market data summary
        display(HTML("<h3>Key Market Data</h3>"))
        indicators = {
            'Price': row['Close'],
            'Daily Change': f"{(row['Close']/row['Close_Shift1'] - 1) * 100:.2f}%" if pd.notna(row['Close_Shift1']) and row['Close_Shift1'] != 0 else "N/A",
            'SMA Fast (20d)': row['SMA_Fast'],
            'SMA Slow (50d)': row['SMA_Slow'],
            'Momentum (63d)': f"{row['Momentum']:.2f}%",
            'Volatility (Ann.)': f"{row['Volatility']:.2f}%",
            'VIX': row['VIX'],
            'Choppiness Index': row['Choppiness_Index'],
        }
        
        indicators_df = pd.DataFrame(list(indicators.items()), columns=['Indicator', 'Value'])
        display(indicators_df)
        
        # Add button to drill down
        drill_button = widgets.Button(
            description='Drill Down to Regime Scores',
            button_style='info',
            tooltip='View detailed regime scores'
        )
        
        def on_drill_button_click(b):
            state['level'] = 2
            exploration_level.value = 2
            update_dashboard()
        
        drill_button.on_click(on_drill_button_click)
        display(drill_button)
    
    def show_level2_regime_comparison(date, row, details):
        """Show comparison of all regime scores"""
        # Create header
        regime = details['winning_regime']
        confidence = details['confidence']
        
        header_html = f"""
        <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px;">
            <h2>Regime Score Comparison: {date.strftime('%Y-%m-%d')}</h2>
            <h3>Current Regime: <span style="color:{colors[regime]['primary']}">{regime}</span> 
            (Confidence: {confidence})</h3>
        </div>
        """
        display(HTML(header_html))
        
        # Create score comparison chart
        fig = go.Figure()
        
        # Raw scores
        raw_scores = [
            details['raw_scores']['Bull'], 
            details['raw_scores']['Neutral'], 
            details['raw_scores']['Bear']
        ]
        
        # Persistence bonuses
        persistence = [
            details['persistence_bonuses']['Bull'],
            details['persistence_bonuses']['Neutral'],
            details['persistence_bonuses']['Bear']
        ]
        
        # Create the bar chart with raw scores and persistence stacked
        fig.add_trace(go.Bar(
            x=['Bull', 'Neutral', 'Bear'],
            y=raw_scores,
            name='Raw Score',
            marker_color=[colors['Bull']['primary'], colors['Neutral']['primary'], colors['Bear']['primary']]
        ))
        
        fig.add_trace(go.Bar(
            x=['Bull', 'Neutral', 'Bear'],
            y=persistence,
            name='Persistence Bonus',
            marker_color=[colors['Bull']['medium'], colors['Neutral']['medium'], colors['Bear']['medium']]
        ))
        
        fig.update_layout(
            title="Regime Scores Breakdown",
            xaxis_title="Regime",
            yaxis_title="Score",
            barmode='stack',
            height=500,
            hovermode="x unified"
        )
        
        # Add a line for the winning threshold
        winning_threshold = details['winning_regime']
        
        # Display the figure
        display(fig)
        
        # Display score summary table
        score_table = pd.DataFrame({
            'Regime': ['Bull', 'Neutral', 'Bear'],
            'Raw Score': [details['raw_scores']['Bull'], details['raw_scores']['Neutral'], details['raw_scores']['Bear']],
            'Persistence Bonus': [details['persistence_bonuses']['Bull'], details['persistence_bonuses']['Neutral'], details['persistence_bonuses']['Bear']],
            'Final Score': [details['final_scores']['Bull'], details['final_scores']['Neutral'], details['final_scores']['Bear']]
        })
        
        # Highlight winning regime
        display(HTML("<h3>Score Summary</h3>"))
        display(score_table)
        
        # Create buttons for drilling down to each regime
        button_container = widgets.HBox([
            widgets.Button(
                description=f'Drill Down to {r} Rules',
                button_style='info',
                layout=widgets.Layout(width='auto')
            ) for r in ['Bull', 'Neutral', 'Bear']
        ])
        
        # Set up button click handlers
        def create_handler(regime):
            def handler(b):
                state['regime'] = regime
                state['level'] = 3
                exploration_level.value = 3
                regime_selector.value = regime
                update_dashboard()
            return handler
        
        for i, regime in enumerate(['Bull', 'Neutral', 'Bear']):
            button_container.children[i].on_click(create_handler(regime))
        
        display(HTML("<h3>Select a Regime to Explore:</h3>"))
        display(button_container)
        
        # Back button
        back_button = widgets.Button(
            description='Back to Overview',
            button_style='warning',
            tooltip='Return to the overview'
        )
        
        def on_back_button_click(b):
            state['level'] = 1
            exploration_level.value = 1
            update_dashboard()
        
        back_button.on_click(on_back_button_click)
        display(back_button)
    
    def show_level2_regime_detail(date, regime_name, row, details):
        """Show detailed breakdown of a specific regime"""
        # Create header with regime info
        raw_score = details['raw_scores'][regime_name]
        persistence = details['persistence_bonuses'][regime_name]
        final_score = details['final_scores'][regime_name]
        is_winning = regime_name == details['winning_regime']
        
        header_html = f"""
        <div style="background-color:{colors[regime_name]['light']}; padding:15px; border-radius:5px; margin-bottom:20px;">
            <h2>{regime_name} Regime Details: {date.strftime('%Y-%m-%d')}</h2>
            <h3>Score: {final_score:.2f} (Raw: {raw_score:.2f} + Persistence: {persistence:.2f})</h3>
            <h4>Status: {"✓ Current Regime" if is_winning else "✗ Not Active"}</h4>
        </div>
        """
        display(HTML(header_html))
        
        # Show regime rules button
        regime_rules_button = widgets.Button(
            description=f'View {regime_name} Rule Details',
            button_style='info',
            tooltip=f'Explore the {regime_name} regime rules'
        )
        
        def on_regime_rules_click(b):
            state['level'] = 3
            exploration_level.value = 3
            update_dashboard()
        
        regime_rules_button.on_click(on_regime_rules_click)
        display(regime_rules_button)
        
        # Navigation buttons
        button_container = widgets.HBox([
            widgets.Button(description='Back to Regime Comparison', button_style='warning'),
            widgets.Button(description='Back to Overview', button_style='warning')
        ])
        
        def back_to_comparison(b):
            state['regime'] = 'Summary'
            regime_selector.value = 'Summary'
            update_dashboard()
        
        def back_to_overview(b):
            state['level'] = 1
            state['regime'] = 'Summary'
            regime_selector.value = 'Summary'
            exploration_level.value = 1
            update_dashboard()
            
        button_container.children[0].on_click(back_to_comparison)
        button_container.children[1].on_click(back_to_overview)
        
        display(button_container)
    
    def show_level3_rule_breakdown(date, regime_name, details):
        """Show breakdown of rules for a specific regime"""
        regime_details = details[regime_name]
        raw_score = details['raw_scores'][regime_name]
        final_score = details['final_scores'][regime_name]
        
        header_html = f"""
        <div style="background-color:{colors[regime_name]['light']}; padding:15px; border-radius:5px; margin-bottom:20px;">
            <h2>{regime_name} Regime Rules: {date.strftime('%Y-%m-%d')}</h2>
            <h3>Total Score: {final_score:.2f} (Raw: {raw_score:.2f})</h3>
        </div>
        """
        display(HTML(header_html))
        
        # Create rule breakdown visualization
        rules_data = []
        
        for rule_id, rule in regime_details.items():
            if rule_id != 'error':  # Skip error entries
                rules_data.append({
                    'Rule ID': rule_id,
                    'Rule Name': rule['name'],
                    'Weight': rule['weight'],
                    'Score': rule['score'],
                    'Passed': rule['passed'],
                    'Description': rule['description']
                })
        
        # Convert to DataFrame and sort by score contribution
        rules_df = pd.DataFrame(rules_data)
        rules_df = rules_df.sort_values('Score', ascending=False)
        
        # Create bar chart of rule contributions
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            y=rules_df['Rule Name'],
            x=rules_df['Score'],
            orientation='h',
            marker_color=[colors[regime_name]['primary'] if passed else 'lightgray' 
                          for passed in rules_df['Passed']],
            text=rules_df['Score'].apply(lambda x: f"{x:.2f}"),
            hovertemplate='%{y}: %{x:.2f}<extra></extra>',
            name='Score'
        ))
        
        fig.update_layout(
            title=f"{regime_name} Regime Rule Contributions",
            xaxis_title="Score Contribution",
            yaxis_title="Rule",
            height=600,
            hoverlabel=dict(bgcolor="white", font_size=12),
            margin=dict(l=200)  # Add more left margin for rule names
        )
        
        display(fig)
        
        # Create interactive rule table
        display(HTML("<h3>Rule Details</h3>"))
        display(HTML("<p>Click on a rule to see detailed breakdown</p>"))
        
        # Use a nicer styled table with click functionality
        table_rows = ""
        for _, rule in rules_df.iterrows():
            status = "✓" if rule['Passed'] else "✗"
            color = colors[regime_name]['primary'] if rule['Passed'] else 'gray'
            row_html = f"""
            <tr class="rule-row" data-rule-id="{rule['Rule ID']}">
                <td style="text-align:center;"><span style="color:{color}; font-size:16px;">{status}</span></td>
                <td><b>{rule['Rule Name']}</b></td>
                <td>{rule['Description']}</td>
                <td style="text-align:right;">{rule['Weight']}</td>
                <td style="text-align:right; font-weight:bold;">{rule['Score']:.2f}</td>
            </tr>
            """
            table_rows += row_html
        
        table_html = f"""
        <style>
        .rule-table {{
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }}
        .rule-table th {{
            background-color: #f5f5f5;
            padding: 8px;
            text-align: left;
            border-bottom: 2px solid #ddd;
        }}
        .rule-table td {{
            padding: 8px;
            border-bottom: 1px solid #ddd;
        }}
        .rule-row {{
            cursor: pointer;
        }}
        .rule-row:hover {{
            background-color: {colors[regime_name]['light']};
        }}
        </style>
        <table class="rule-table" id="regime-rule-table">
            <thead>
                <tr>
                    <th style="width:5%;">Status</th>
                    <th style="width:15%;">Rule</th>
                    <th style="width:50%;">Description</th>
                    <th style="width:15%; text-align:right;">Weight</th>
                    <th style="width:15%; text-align:right;">Score</th>
                </tr>
            </thead>
            <tbody>
                {table_rows}
            </tbody>
        </table>
        <script>
        // Add click handler for rule rows
        document.querySelectorAll('.rule-row').forEach(function(row) {{
            row.addEventListener('click', function() {{
                var ruleId = this.getAttribute('data-rule-id');
                // Call Python function via comm
                IPython.notebook.kernel.execute(`rule_selected("${ruleId}")`);
            }});
        }});
        </script>
        """
        
        display(HTML(table_html))
        
        # Set up the callback for rule selection
        def rule_selected(rule_id):
            state['selected_rule'] = rule_id
            state['level'] = 4
            exploration_level.value = 4
            update_dashboard()
        
        # Make the callback available to JavaScript
        import IPython
        IPython.display.display_html(
            IPython.display.Javascript("""
            window.rule_selected = function(rule_id) {
                IPython.notebook.kernel.execute(
                    "state['selected_rule'] = '" + rule_id + "'; " +
                    "state['level'] = 4; " +
                    "exploration_level.value = 4; " +
                    "update_dashboard()"
                );
            }
            """),
            raw=True
        )
        
        # Navigation buttons
        button_container = widgets.HBox([
            widgets.Button(description='Back to Regime Comparison', button_style='warning'),
            widgets.Button(description='Back to Overview', button_style='warning')
        ])
        
        def back_to_comparison(b):
            state['regime'] = 'Summary'
            state['level'] = 2
            regime_selector.value = 'Summary'
            exploration_level.value = 2
            update_dashboard()
        
        def back_to_overview(b):
            state['level'] = 1
            state['regime'] = 'Summary'
            regime_selector.value = 'Summary'
            exploration_level.value = 1
            update_dashboard()
            
        button_container.children[0].on_click(back_to_comparison)
        button_container.children[1].on_click(back_to_overview)
        
        display(button_container)
    
    def show_level4_rule_detail(date, regime_name, rule_id, details):
        """Show detailed breakdown of a specific rule"""
        regime_details = details[regime_name]
        
        if rule_id not in regime_details:
            display(HTML(f"<h3>Rule details for {rule_id} not found</h3>"))
            return
            
        rule = regime_details[rule_id]
        
        # Create header
        header_html = f"""
        <div style="background-color:{colors[regime_name]['light']}; padding:15px; border-radius:5px; margin-bottom:20px;">
            <h2>{rule['name']} Rule Detail</h2>
            <h3>{regime_name} Regime - {date.strftime('%Y-%m-%d')}</h3>
            <h4>Status: <span style="color:{colors[regime_name]['primary'] if rule['passed'] else 'gray'}">
                {"✓ Passed" if rule['passed'] else "✗ Failed"}</span>
            </h4>
        </div>
        """
        display(HTML(header_html))
        
        # Display rule description and values
        description_html = f"""
        <div style="margin-bottom:20px;">
            <h3>Rule Description</h3>
            <p style="font-size:16px;">{rule['description']}</p>
            
            <h3>Scoring</h3>
            <ul>
                <li><b>Weight:</b> {rule['weight']}</li>
                <li><b>Score:</b> {rule['score']:.2f}</li>
                <li><b>Contribution:</b> {(rule['score'] / details['raw_scores'][regime_name] * 100):.1f}% of raw {regime_name} score</li>
            </ul>
        </div>
        """
        display(HTML(description_html))
        
        # Display values used in calculation
        display(HTML("<h3>Values Used</h3>"))
        
        if 'values' in rule:
            values = rule['values']
            values_table = []
            
            for key, value in values.items():
                if isinstance(value, (int, float)) and not isinstance(value, bool):
                    # Format numbers nicely
                    if abs(value) < 0.01 or abs(value) >= 10000:
                        formatted_value = f"{value:.6g}"
                    else:
                        formatted_value = f"{value:.4f}"
                else:
                    formatted_value = str(value)
                    
                values_table.append({
                    'Parameter': key,
                    'Value': formatted_value
                })
            
            values_df = pd.DataFrame(values_table)
            display(values_df)
            
            # Create visualization for rule calculation if possible
            try:
                if rule_id.startswith('rule') and any(x in rule['description'].lower() for x in ['threshold', 'percentile', 'z']):
                    display(HTML("<h3>Rule Visualization</h3>"))
                    
                    # Different visualizations based on rule type
                    if 'threshold' in rule['description'].lower():
                        # Find main value and threshold
                        main_value = None
                        threshold = None
                        is_greater = '>' in rule['description']
                        
                        # Simple heuristic detection
                        for k, v in values.items():
                            if isinstance(v, (int, float)) and 'threshold' in k.lower():
                                threshold = v
                            elif isinstance(v, (int, float)) and not isinstance(v, bool) and 'threshold' not in k.lower():
                                if main_value is None:  # Take the first numeric non-threshold as main value
                                    main_value = v
                        
                        if main_value is not None and threshold is not None:
                            fig = go.Figure()
                            
                            # Create threshold visualization
                            if is_greater:
                                # Value needs to be > threshold
                                fig.add_shape(
                                    type="rect",
                                    x0=0, y0=threshold,
                                    x1=1, y1=max(main_value * 1.2, threshold * 1.2),
                                    fillcolor="rgba(0, 255, 0, 0.2)",
                                    line=dict(width=0),
                                    layer="below"
                                )
                                fig.add_shape(
                                    type="rect",
                                    x0=0, y0=0,
                                    x1=1, y1=threshold,
                                    fillcolor="rgba(255, 0, 0, 0.2)",
                                    line=dict(width=0),
                                    layer="below"
                                )
                            else:
                                # Value needs to be < threshold
                                fig.add_shape(
                                    type="rect",
                                    x0=0, y0=0,
                                    x1=1, y1=threshold,
                                    fillcolor="rgba(0, 255, 0, 0.2)",
                                    line=dict(width=0),
                                    layer="below"
                                )
                                fig.add_shape(
                                    type="rect",
                                    x0=0, y0=threshold,
                                    x1=1, y1=max(main_value * 1.2, threshold * 1.2),
                                    fillcolor="rgba(255, 0, 0, 0.2)",
                                    line=dict(width=0),
                                    layer="below"
                                )
                            
                            # Add threshold line
                            fig.add_shape(
                                type="line",
                                x0=0, y0=threshold,
                                x1=1, y1=threshold,
                                line=dict(color="black", width=2, dash="dash")
                            )
                            
                            # Add the actual value
                            fig.add_trace(go.Scatter(
                                x=[0.5],
                                y=[main_value],
                                mode='markers+text',
                                marker=dict(
                                    size=15,
                                    color=colors[regime_name]['primary'] if rule['passed'] else 'gray'
                                ),
                                text=['Current Value'],
                                textposition="top center"
                            ))
                            
                            fig.update_layout(
                                title="Threshold Comparison",
                                xaxis=dict(
                                    showticklabels=False,
                                    showgrid=False,
                                    zeroline=False
                                ),
                                yaxis_title="Value",
                                height=400,
                                showlegend=False,
                                annotations=[
                                    dict(
                                        x=0.5, y=threshold,
                                        xref="paper", yref="y",
                                        text=f"Threshold: {threshold}",
                                        showarrow=False,
                                        yanchor="bottom" if is_greater else "top",
                                        bgcolor="white",
                                        borderpad=4
                                    )
                                ]
                            )
                            
                            display(fig)
            except Exception as e:
                display(HTML(f"<p>Could not create visualization: {str(e)}</p>"))
        
        # Navigation buttons
        button_container = widgets.HBox([
            widgets.Button(description='Back to Rule List', button_style='warning'),
            widgets.Button(description='Back to Overview', button_style='warning')
        ])
        
        def back_to_rules(b):
            state['level'] = 3
            state['selected_rule'] = None
            exploration_level.value = 3
            update_dashboard()
        
        def back_to_overview(b):
            state['level'] = 1
            state['regime'] = 'Summary'
            state['selected_rule'] = None
            regime_selector.value = 'Summary'
            exploration_level.value = 1
            update_dashboard()
            
        button_container.children[0].on_click(back_to_rules)
        button_container.children[1].on_click(back_to_overview)
        
        display(button_container)
    
    # Set up event handlers for controls
    def on_date_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            state['date'] = change['new']
            update_dashboard()
    
    def on_regime_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            state['regime'] = change['new']
            update_dashboard()
    
    def on_level_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            state['level'] = change['new']
            update_dashboard()
    
    date_picker.observe(on_date_change, names='value')
    regime_selector.observe(on_regime_change, names='value')
    exploration_level.observe(on_level_change, names='value')
    
    # Create layout
    controls = widgets.HBox([date_picker, regime_selector, exploration_level])
    layout = widgets.VBox([controls, level_descriptions, dashboard_output])
    
    # Initial display
    display(layout)
    update_dashboard()
    
    return layout

# Call this function in a separate cell
create_market_regime_dashboard()
